![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader
from sparknlp.pretrained import  PretrainedPipeline

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version :  5.1.0
sparknlp_jsl.version :  5.1.0


## Pretrained Relation Extraction Models and Pipelines

Here are the list of pretrained Relation Extraction models and pipelines:
  
  *  **Relation Extraction Models**

|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [generic_re](https://nlp.johnsnowlabs.com/2022/12/20/generic_re.html)  | 2| [posology_re](https://nlp.johnsnowlabs.com/2020/09/01/posology_re.html)  | 3| [re_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/16/re_ade_biobert_en.html)  |
| 4| [re_ade_clinical](https://nlp.johnsnowlabs.com/2021/07/12/re_ade_clinical_en.html)  | 5| [re_ade_conversational](https://nlp.johnsnowlabs.com/2022/07/27/re_ade_conversational_en_3_0.html)  | 6| [re_bodypart_directions](https://nlp.johnsnowlabs.com/2021/01/18/re_bodypart_directions_en.html)  |
| 7| [re_bodypart_problem](https://nlp.johnsnowlabs.com/2021/01/18/re_bodypart_problem_en.html)  | 8| [re_bodypart_proceduretest](https://nlp.johnsnowlabs.com/2021/01/18/re_bodypart_proceduretest_en.html)  | 9| [re_clinical](https://nlp.johnsnowlabs.com/2020/09/24/re_clinical_en.html)  |
| 10| [re_date_clinical](https://nlp.johnsnowlabs.com/2021/01/18/re_date_clinical_en.html)  | 11| [re_drug_drug_interaction_clinical](https://nlp.johnsnowlabs.com/2020/09/03/re_drug_drug_interaction_clinical_en.html)  | 12| [re_drugprot_clinical](https://nlp.johnsnowlabs.com/2022/01/05/re_drugprot_clinical_en.html)  |
| 13| [re_human_phenotype_gene_clinical](https://nlp.johnsnowlabs.com/2020/09/30/re_human_phenotype_gene_clinical_en.html)  | 14| [re_oncology_biomarker_result_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_biomarker_result_wip_en.html)  | 15| [re_oncology_granular_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_granular_wip_en.html)  |
| 16| [re_oncology_location_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_location_wip_en.html)  | 17| [re_oncology_size_wip](https://nlp.johnsnowlabs.com/2022/09/26/re_oncology_size_wip_en.html)  | 18| [re_oncology_temporal_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_temporal_wip_en.html)  |
| 19| [re_oncology_test_result_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_test_result_wip_en.html)  | 20| [re_oncology_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_wip_en.html)  | 21| [re_temporal_events_clinical](https://nlp.johnsnowlabs.com/2020/09/28/re_temporal_events_clinical_en.html)  |
| 22| [re_temporal_events_enriched_clinical](https://nlp.johnsnowlabs.com/2020/09/28/re_temporal_events_enriched_clinical_en.html)  | 23| [re_test_problem_finding](https://nlp.johnsnowlabs.com/2021/04/19/re_test_problem_finding_en.html)  | 24| [re_test_result_date](https://nlp.johnsnowlabs.com/2021/02/24/re_test_result_date_en.html)  |
| 25| [re_zeroshot_biobert](https://nlp.johnsnowlabs.com/2022/04/05/re_zeroshot_biobert_en_3_0.html)  | 26| [redl_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/12/redl_ade_biobert_en.html)  | 27| [redl_bodypart_direction_biobert](https://nlp.johnsnowlabs.com/2021/06/01/redl_bodypart_direction_biobert_en.html)  |
| 28| [redl_bodypart_problem_biobert](https://nlp.johnsnowlabs.com/2021/06/01/redl_bodypart_problem_biobert_en.html)  | 29| [redl_bodypart_procedure_test_biobert](https://nlp.johnsnowlabs.com/2021/09/10/redl_bodypart_procedure_test_biobert_en.html)  | 30| [redl_chemprot_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_chemprot_biobert_en.html)  |
| 31| [redl_clinical_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_clinical_biobert_en.html)  | 32| [redl_date_clinical_biobert](https://nlp.johnsnowlabs.com/2021/06/01/redl_date_clinical_biobert_en.html)  | 33| [redl_drug_drug_interaction_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_drug_drug_interaction_biobert_en.html)  |
| 34| [redl_drugprot_biobert](https://nlp.johnsnowlabs.com/2023/01/14/redl_drugprot_biobert_en.html)  | 35| [redl_human_phenotype_gene_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_human_phenotype_gene_biobert_en.html)  | 36| [redl_nihss_biobert](https://nlp.johnsnowlabs.com/2021/11/16/redl_nihss_biobert_en.html)  |
| 37| [redl_oncology_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_biobert_wip_en.html)  | 38| [redl_oncology_biomarker_result_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_biomarker_result_biobert_wip_en.html)  | 39| [redl_oncology_granular_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_granular_biobert_wip_en.html)  |
| 40| [redl_oncology_location_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_location_biobert_wip_en.html)  | 41| [redl_oncology_size_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/28/redl_oncology_size_biobert_wip_en.html)  | 42| [redl_oncology_temporal_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_temporal_biobert_wip_en.html)  |
| 43| [redl_oncology_test_result_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_test_result_biobert_wip_en.html)  | 44| [redl_temporal_events_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_temporal_events_biobert_en.html)  | 45| []()|


*  **Relation Extraction Pipelines**

|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [re_bodypart_directions_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_bodypart_directions_pipeline_en_3_0.html)  | 4| [re_human_phenotype_gene_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_human_phenotype_gene_clinical_pipeline_en_3_0.html)  | 7| [re_test_problem_finding_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_test_problem_finding_pipeline_en_3_0.html)  |
| 2| [re_bodypart_proceduretest_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_bodypart_proceduretest_pipeline_en_3_0.html)  | 5| [re_temporal_events_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_temporal_events_clinical_pipeline_en_3_0.html)  | 8| [re_test_result_date_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_test_result_date_pipeline_en_3_0.html)  |
| 3| [re_date_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_date_clinical_pipeline_en_3_0.html)  | 6| [re_temporal_events_enriched_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_temporal_events_enriched_clinical_pipeline_en_3_0.html)  | |

**You can find all these models/pipelines and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?edition=Spark+NLP+for+Healthcare&task=Relation+Extraction)**

## Posology Releation Extraction

This is a demonstration of using SparkNLP for extracting posology relations. The following relatios are supported:

DRUG-DOSAGE
DRUG-FREQUENCY
DRUG-ADE (Adversed Drug Events)
DRUG-FORM
DRUG-ROUTE
DRUG-DURATION
DRUG-REASON
DRUG=STRENGTH

The model has been validated against the posology dataset described in (Magge, Scotch, & Gonzalez-Hernandez, 2018).

| Relation | Recall | Precision | F1 | F1 (Magge, Scotch, & Gonzalez-Hernandez, 2018) |
| --- | --- | --- | --- | --- |
| DRUG-ADE | 0.66 | 1.00 | **0.80** | 0.76 |
| DRUG-DOSAGE | 0.89 | 1.00 | **0.94** | 0.91 |
| DRUG-DURATION | 0.75 | 1.00 | **0.85** | 0.92 |
| DRUG-FORM | 0.88 | 1.00 | **0.94** | 0.95* |
| DRUG-FREQUENCY | 0.79 | 1.00 | **0.88** | 0.90 |
| DRUG-REASON | 0.60 | 1.00 | **0.75** | 0.70 |
| DRUG-ROUTE | 0.79 | 1.00 | **0.88** | 0.95* |
| DRUG-STRENGTH | 0.95 | 1.00 | **0.98** | 0.97 |


*Magge, Scotch, Gonzalez-Hernandez (2018) collapsed DRUG-FORM and DRUG-ROUTE into a single relation.

In [0]:
import functools 
import numpy as np
from scipy import spatial
import pyspark.sql.functions as F
import pyspark.sql.types as T
from sparknlp.base import *

**Build pipeline using SparNLP pretrained models and the relation extration model optimized for posology**.
 
 The precision of the RE model is controlled by "setMaxSyntacticDistance(4)", which sets the maximum syntactic distance between named entities to 4. A larger value will improve recall at the expense at lower precision. A value of 4 leads to literally perfect precision (i.e. the model doesn't produce any false positives) and reasonably good recall.

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = sparknlp.annotators.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

ner_tagger = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

ner_chunker = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reModel = RelationExtractionModel()\
    .pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
ner_posology download started this may take some time.
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]


**Create a light pipeline for annotating free text**

In [0]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also 
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 
12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
"""

lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [0]:
results[0].keys()

dict_keys(['sentences', 'document', 'ner_chunks', 'ner_tags', 'relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [0]:
results[0]['ner_chunks']

[Annotation(chunk, 28, 33, 1 unit, {'chunk': '0', 'confidence': '0.71675', 'ner_source': 'ner_chunks', 'entity': 'DOSAGE', 'sentence': '0'}, []),
 Annotation(chunk, 38, 42, Advil, {'chunk': '1', 'confidence': '0.9984', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '0'}, []),
 Annotation(chunk, 44, 53, for 5 days, {'chunk': '2', 'confidence': '0.7455', 'ner_source': 'ner_chunks', 'entity': 'DURATION', 'sentence': '0'}, []),
 Annotation(chunk, 96, 101, 1 unit, {'chunk': '3', 'confidence': '0.72360003', 'ner_source': 'ner_chunks', 'entity': 'DOSAGE', 'sentence': '1'}, []),
 Annotation(chunk, 106, 114, Metformin, {'chunk': '4', 'confidence': '0.9998', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '1'}, []),
 Annotation(chunk, 116, 120, daily, {'chunk': '5', 'confidence': '0.9997', 'ner_source': 'ner_chunks', 'entity': 'FREQUENCY', 'sentence': '1'}, []),
 Annotation(chunk, 190, 197, 40 units, {'chunk': '6', 'confidence': '0.84085', 'ner_source': 'ner_chunks', 'entity

In [0]:
results[0]['relations']

[Annotation(category, 28, 42, DOSAGE-DRUG, {'chunk2': 'Advil', 'confidence': '1.0', 'entity2_end': '42', 'chunk1': '1 unit', 'entity1': 'DOSAGE', 'entity2_begin': '38', 'chunk2_confidence': '0.9984', 'entity1_begin': '28', 'sentence': '0', 'direction': 'both', 'entity1_end': '33', 'entity2': 'DRUG', 'chunk1_confidence': '0.71675'}, []),
 Annotation(category, 38, 53, DRUG-DURATION, {'chunk2': 'for 5 days', 'confidence': '1.0', 'entity2_end': '53', 'chunk1': 'Advil', 'entity1': 'DRUG', 'entity2_begin': '44', 'chunk2_confidence': '0.7455', 'entity1_begin': '38', 'sentence': '0', 'direction': 'both', 'entity1_end': '42', 'entity2': 'DURATION', 'chunk1_confidence': '0.9984'}, []),
 Annotation(category, 96, 114, DOSAGE-DRUG, {'chunk2': 'Metformin', 'confidence': '1.0', 'entity2_end': '114', 'chunk1': '1 unit', 'entity1': 'DOSAGE', 'entity2_begin': '106', 'chunk2_confidence': '0.9998', 'entity1_begin': '96', 'sentence': '1', 'direction': 'both', 'entity1_end': '101', 'entity2': 'DRUG', 'chunk

**Show extracted relations**

In [0]:
for rel in results[0]["relations"]:
    print("{}({}={} - {}={})".format(
        rel.result,
        rel.metadata['entity1'], 
        rel.metadata['chunk1'], 
        rel.metadata['entity2'],
        rel.metadata['chunk2']
    ))

DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Advil)
DRUG-DURATION(DRUG=Advil - DURATION=for 5 days)
DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Metformin)
DRUG-FREQUENCY(DRUG=Metformin - FREQUENCY=daily)
DOSAGE-DRUG(DOSAGE=40 units - DRUG=insulin glargine)
DRUG-FREQUENCY(DRUG=insulin glargine - FREQUENCY=at night)
DOSAGE-DRUG(DOSAGE=12 units - DRUG=insulin lispro)
DRUG-FREQUENCY(DRUG=insulin lispro - FREQUENCY=with meals)
DRUG-STRENGTH(DRUG=metformin - STRENGTH=1000 mg)
DRUG-FREQUENCY(DRUG=metformin - FREQUENCY=two times a day)


In [0]:
results[0]["relations"]

[Annotation(category, 28, 42, DOSAGE-DRUG, {'chunk2': 'Advil', 'confidence': '1.0', 'entity2_end': '42', 'chunk1': '1 unit', 'entity1': 'DOSAGE', 'entity2_begin': '38', 'chunk2_confidence': '0.9984', 'entity1_begin': '28', 'sentence': '0', 'direction': 'both', 'entity1_end': '33', 'entity2': 'DRUG', 'chunk1_confidence': '0.71675'}, []),
 Annotation(category, 38, 53, DRUG-DURATION, {'chunk2': 'for 5 days', 'confidence': '1.0', 'entity2_end': '53', 'chunk1': 'Advil', 'entity1': 'DRUG', 'entity2_begin': '44', 'chunk2_confidence': '0.7455', 'entity1_begin': '38', 'sentence': '0', 'direction': 'both', 'entity1_end': '42', 'entity2': 'DURATION', 'chunk1_confidence': '0.9984'}, []),
 Annotation(category, 96, 114, DOSAGE-DRUG, {'chunk2': 'Metformin', 'confidence': '1.0', 'entity2_end': '114', 'chunk1': '1 unit', 'entity1': 'DOSAGE', 'entity2_begin': '106', 'chunk2_confidence': '0.9998', 'entity1_begin': '96', 'sentence': '1', 'direction': 'both', 'entity1_end': '101', 'entity2': 'DRUG', 'chunk

In [0]:
import pandas as pd

# get relations in a pandas dataframe

def get_relations_df (results, rel_col='relations', chunk_col='ner_chunks'):
    rel_pairs=[]
    chunks = []

    for rel in results[0][rel_col]:
        rel_pairs.append((
            rel.metadata['entity1_begin'],
            rel.metadata['entity1_end'],
            rel.metadata['chunk1'], 
            rel.metadata['entity1'], 
            rel.metadata['entity2_begin'],
            rel.metadata['entity2_end'],
            rel.metadata['chunk2'], 
            rel.metadata['entity2'],
            rel.result, 
            rel.metadata['confidence'],
        ))

    for chunk in results[0][chunk_col]:
        chunks.append((
            chunk.metadata["sentence"],
            chunk.begin,
            chunk.end,
            chunk.result, 
        ))

    rel_df = pd.DataFrame(rel_pairs, columns=['entity1_begin', 'entity1_end', 'chunk1', 'entity1', 'entity2_begin', 'entity2_end', 'chunk2', 'entity2', 'relation', 'confidence'])

    chunks_df = pd.DataFrame(chunks, columns = ["sentence", "begin", "end", "chunk"])
    chunks_df.begin = chunks_df.begin.astype(str)
    chunks_df.end = chunks_df.end.astype(str)

    result_df = pd.merge(rel_df,chunks_df, left_on=["entity1_begin", "entity1_end", "chunk1"], right_on=["begin", "end", "chunk"])[["sentence"] + list(rel_df.columns)]


    return result_df

In [0]:
print(text, "\n")

rel_df = get_relations_df (results)
rel_df


The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also 
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 
12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
 



,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,28,33,1 unit,DOSAGE,38,42,Advil,DRUG,DOSAGE-DRUG,1.0
1,0,38,42,Advil,DRUG,44,53,for 5 days,DURATION,DRUG-DURATION,1.0
2,1,96,101,1 unit,DOSAGE,106,114,Metformin,DRUG,DOSAGE-DRUG,1.0
3,1,106,114,Metformin,DRUG,116,120,daily,FREQUENCY,DRUG-FREQUENCY,1.0
4,2,190,197,40 units,DOSAGE,202,217,insulin glargine,DRUG,DOSAGE-DRUG,1.0
5,2,202,217,insulin glargine,DRUG,219,226,at night,FREQUENCY,DRUG-FREQUENCY,1.0
6,2,231,238,12 units,DOSAGE,243,256,insulin lispro,DRUG,DOSAGE-DRUG,1.0
7,2,243,256,insulin lispro,DRUG,258,267,with meals,FREQUENCY,DRUG-FREQUENCY,1.0
8,2,275,283,metformin,DRUG,285,291,1000 mg,STRENGTH,DRUG-STRENGTH,1.0
9,2,275,283,metformin,DRUG,293,307,two times a day,FREQUENCY,DRUG-FREQUENCY,1.0


In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""

annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df



,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,1,493,500,five-day,DURATION,512,522,amoxicillin,DRUG,DURATION-DRUG,1.0
1,3,681,693,dapagliflozin,DRUG,695,708,for six months,DURATION,DRUG-DURATION,1.0
2,12,1940,1946,insulin,DRUG,1948,1951,drip,ROUTE,DRUG-ROUTE,1.0
3,14,2255,2262,40 units,DOSAGE,2267,2282,insulin glargine,DRUG,DOSAGE-DRUG,1.0
4,14,2267,2282,insulin glargine,DRUG,2284,2291,at night,FREQUENCY,DRUG-FREQUENCY,1.0
5,14,2295,2302,12 units,DOSAGE,2307,2320,insulin lispro,DRUG,DOSAGE-DRUG,1.0
6,14,2307,2320,insulin lispro,DRUG,2322,2331,with meals,FREQUENCY,DRUG-FREQUENCY,1.0
7,14,2339,2347,metformin,DRUG,2349,2355,1000 mg,STRENGTH,DRUG-STRENGTH,1.0
8,14,2339,2347,metformin,DRUG,2357,2371,two times a day,FREQUENCY,DRUG-FREQUENCY,1.0


**Visualization of Extracted Relations**

In [0]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also 
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine , 
12 units of insulin lispro with meals , and metformin two times a day.
"""

lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [0]:
from sparknlp_display import RelationExtractionVisualizer


visualizer = RelationExtractionVisualizer()
vis = visualizer.display(results[0], 'relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

The patient was prescribed 1 unit DOSAGE of Advil DRUG for 5 days DURATION after meals. The patient was also given 1 unit DOSAGE of Metformin DRUG daily FREQUENCY . He was seen by the endocrinology service and she was discharged on 40 units DOSAGE of insulin glargine DRUG , 12 units DOSAGE of insulin lispro DRUG with meals FREQUENCY , and metformin DRUG two times a day FREQUENCY . <polyline fill="none" points="246.0,155.0 246.8877551020408,153.7960798646822 247.7755102040816,152.59312871337622 248.6632653061225,151.39211553009378 249.55102040816325,150.19400929884657 250.4387755102041,148.99977900364644 251.32653061224488,147.81039362850512 252.21428571428575,146.62682215743445 253.10204081632654,145.45003357444605 253.98979591836738,144.28099686355176 254.87755102040813,143.12068100876337 255.76530612244895,141.97005499409258 256.65306122448976,140.83008780355124 257.5408163265306,139.70174842115105 258.4285714285714,138.5860058309038 259.3163265306123,137.48382901682123 260.2040816326531,136.39618696291512 261.09183673469397,135.32404865319725 261.9795918367347,134.2683830716793 262.8673469387755,133.23015920237316 263.75510204081627,132.2103460292905 264.6428571428571,131.20991253644314 265.53061224489795,130.22982770784282 266.4183673469388,129.2710605275013 267.3061224489796,128.33457997943034 268.19387755102036,127.42135504764173 269.0816326530612,126.53235471614718 269.96938775510205,125.6685479689585 270.8571428571429,124.83090379008749 271.7448979591837,124.02039116354581 272.63265306122446,123.2379790733453 273.5204081632653,122.48463650349771 274.40816326530614,121.76133243801479 275.2959183673469,121.06903586090831 276.1836734693877,120.40871575619002 277.07142857142856,119.78134110787173 277.9591836734694,119.18788089996517 278.8469387755102,118.62930411648207 279.734693877551,118.10657974143427 280.6224489795918,117.62067675883348 281.51020408163265,117.17256415269148 282.3979591836735,116.76321090702004 283.2857142857143,116.39358600583091 284.17346938775506,116.06465843313586 285.0612244897959,115.77739717294665 285.94897959183675,115.53277120927505 286.8367346938776,115.33174952613282 287.7244897959184,115.17530110753174 288.6122448979592,115.06439493748353 289.5,115.0 289.5,115.0 290.38775510204084,114.9827622844223 291.27551020408157,115.01203580140927 292.1632653061224,115.08685156694916 293.0510204081633,115.20624059703015 293.9387755102041,115.36923390764053 294.8265306122448,115.57486251476851 295.7142857142857,115.82215743440234 296.60204081632656,116.11014968253025 297.48979591836735,116.43787027514047 298.37755102040813,116.80435022822121 299.2653061224489,117.20862055776078 300.15306122448976,117.64971227974738 301.04081632653055,118.12665641016923 301.92857142857144,118.63848396501459 302.8163265306123,119.18422596027166 303.7040816326531,119.7629134119287 304.5918367346939,120.37357733597398 305.4795918367347,121.01524874839565 306.3673469387755,121.68695866518203 307.25510204081627,122.3877381023213 308.1428571428571,123.11661807580174 309.03061224489795,123.87262960161156 309.91836734693874,124.654803695739 310.80612244897964,125.46217137417233 311.6938775510204,126.29376365289971 312.5816326530612,127.14861154790944 313.469387755102,128.02574607518974 314.3571428571429,128.92419825072886 315.24489795918373,129.842999090515 316.13265306122446,130.7811796105364 317.0204081632653,131.73777082678137 317.9081632653061,132.71180375523804 318.7959183673469,133.7023094118947 319.6836734693877,134.7083188127396 320.57142857142856,135.72886297376095 321.4591836734694,136.76297291094698 322.34693877551024,137.80967964028594 323.234693877551,138.86801417776604 324.1224489795918,139.9370075393756 325.01020408163265,141.01569074110276 325.8979591836735,142.1030947989358 326.78571428571433,143.19825072886297 327.67346938775506,144.30018954687245 328.5612244897959,145.40794226895255 329.44897959183675,146.52053991109145 330.33673469387753,147.63701348927742 331.2244897959184,148.75639401949866 332.1122448979591

## ADE Relation Extraction

We can extract relations between `DRUG` and adverse reactions `ADE` caused by them. Also we can find if an adverse event is caused by a drug or not.

`1`: Shows the adverse event and drug entities are related.

`0`: Shows the adverse event and drug entities are not related.

Now we will first use `ner_ade_clinical` NER model and detect `DRUG` and `ADE` entities. Then we ca find the relations between them by using `re_ade_clinical` model. We can also use the same pipeline elemnets rest of them we created above.

In [0]:
ner_tagger = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")  

reModel = RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setRelationPairsCaseSensitive(False)     # it will return any "ade-drug" or "ADE-DRUG" relationship. By default it’s set to False
                                              # True, then the pairs of entities in the dataset should match the pairs in setRelationPairs in their specific case (case sensitive). 
                                              # False, meaning that the match of those relation names is case insensitive.


ade_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_model = ade_pipeline.fit(empty_data)

ner_ade_clinical download started this may take some time.
[OK!]
re_ade_clinical download started this may take some time.
[OK!]


**Show the classes**

In [0]:
reModel.getClasses()

['0', '1']

**We will create a LightPipeline for annotation.**

In [0]:
ade_lmodel = LightPipeline(ade_model)

In [0]:
text = "I experienced fatigue, muscle cramps, anxiety, agression and sadness after taking Lipitor but no more adverse after passing Zocor."


ade_results = ade_lmodel.fullAnnotate(text)

In [0]:
ade_results[0].keys()

dict_keys(['sentences', 'document', 'ner_chunks', 'ner_tags', 'relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [0]:
ade_results[0]["ner_chunks"]

[Annotation(chunk, 14, 20, fatigue, {'chunk': '0', 'confidence': '0.8692', 'ner_source': 'ner_chunks', 'entity': 'ADE', 'sentence': '0'}, []),
 Annotation(chunk, 23, 35, muscle cramps, {'chunk': '1', 'confidence': '0.9433', 'ner_source': 'ner_chunks', 'entity': 'ADE', 'sentence': '0'}, []),
 Annotation(chunk, 38, 44, anxiety, {'chunk': '2', 'confidence': '0.9881', 'ner_source': 'ner_chunks', 'entity': 'ADE', 'sentence': '0'}, []),
 Annotation(chunk, 47, 55, agression, {'chunk': '3', 'confidence': '0.9664', 'ner_source': 'ner_chunks', 'entity': 'ADE', 'sentence': '0'}, []),
 Annotation(chunk, 61, 67, sadness, {'chunk': '4', 'confidence': '0.9415', 'ner_source': 'ner_chunks', 'entity': 'ADE', 'sentence': '0'}, []),
 Annotation(chunk, 82, 88, Lipitor, {'chunk': '5', 'confidence': '0.9945', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '0'}, []),
 Annotation(chunk, 124, 128, Zocor, {'chunk': '6', 'confidence': '0.9271', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': 

**Lets show the `ADE`-`DRUG` relations by using pandas dataframe.**

In [0]:
print(text)

rel_df = get_relations_df (ade_results)

rel_df


I experienced fatigue, muscle cramps, anxiety, agression and sadness after taking Lipitor but no more adverse after passing Zocor.


,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,14,20,fatigue,ADE,82,88,Lipitor,DRUG,1,0.9998622
1,0,14,20,fatigue,ADE,124,128,Zocor,DRUG,0,0.99888533
2,0,23,35,muscle cramps,ADE,82,88,Lipitor,DRUG,1,0.99999154
3,0,23,35,muscle cramps,ADE,124,128,Zocor,DRUG,0,0.9780189
4,0,38,44,anxiety,ADE,82,88,Lipitor,DRUG,1,0.86541504
5,0,38,44,anxiety,ADE,124,128,Zocor,DRUG,0,0.99998915
6,0,47,55,agression,ADE,82,88,Lipitor,DRUG,1,0.99999785
7,0,47,55,agression,ADE,124,128,Zocor,DRUG,0,0.999469
8,0,61,67,sadness,ADE,82,88,Lipitor,DRUG,1,1.0
9,0,61,67,sadness,ADE,124,128,Zocor,DRUG,0,0.99980325


**Visualization of relations**

In [0]:
visualizer = RelationExtractionVisualizer()
vis = visualizer.display(ade_results[0], 'relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

I experienced fatigue ADE , muscle cramps ADE , anxiety ADE , agression ADE and sadness ADE after taking Lipitor DRUG but no more adverse after passing Zocor DRUG . <polyline fill="none" points="509.0,90.0 508.69387755102036,90.66326530612244 508.3877551020407,91.32653061224487 508.08163265306126,91.98979591836735 507.7755102040816,92.6530612244898 507.46938775510205,93.31632653061226 507.1632653061224,93.97959183673468 506.857142857143,94.64285714285717 506.5510204081633,95.30612244897961 506.2448979591837,95.96938775510206 505.9387755102041,96.63265306122447 505.6326530612244,97.29591836734693 505.3265306122448,97.95918367346938 505.0204081632653,98.62244897959184 504.7142857142857,99.28571428571429 504.4081632653062,99.94897959183676 504.10204081632656,100.6122448979592 503.795918367347,101.27551020408167 503.48979591836735,101.93877551020408 503.1836734693877,102.60204081632654 502.8775510204081,103.26530612244898 502.5714285714285,103.92857142857142 502.26530612244903,104.59183673469389 501.95918367346934,105.25510204081633 501.6530612244898,105.9183673469388 501.34693877551024,106.58163265306125 501.0408163265306,107.24489795918367 500.734693877551,107.90816326530613 500.42857142857144,108.57142857142858 500.1224489795919,109.23469387755104 499.81632653061223,109.89795918367348 499.5102040816327,110.56122448979593 499.2040816326531,111.22448979591837 498.8979591836735,111.88775510204083 498.5918367346939,112.55102040816327 498.28571428571433,113.21428571428572 497.9795918367347,113.87755102040816 497.6734693877551,114.54081632653063 497.36734693877554,115.20408163265307 497.0612244897959,115.86734693877551 496.7551020408164,116.53061224489795 496.44897959183675,117.19387755102042 496.14285714285717,117.85714285714286 495.8367346938776,118.5204081632653 495.53061224489795,119.18367346938776 495.2244897959184,119.84693877551021 494.9183673469388,120.51020408163265 494.61224489795916,121.1734693877551 494.3061224489796,121.83673469387756 494.0,122.5 494.0,122.5 493.69387755102036,123.16326530612244 493.3877551020407,123.82653061224487 493.08163265306126,124.48979591836735 492.77551020408157,125.1530612244898 492.46938775510205,125.81632653061224 492.16326530612247,126.47959183673467 491.85714285714295,127.14285714285717 491.5510204081633,127.8061224489796 491.24489795918373,128.46938775510205 490.93877551020404,129.13265306122446 490.63265306122446,129.7959183673469 490.3265306122448,130.45918367346937 490.02040816326524,131.1224489795918 489.7142857142857,131.78571428571428 489.4081632653062,132.44897959183675 489.1020408163265,133.1122448979592 488.795918367347,133.77551020408166 488.48979591836735,134.43877551020407 488.18367346938777,135.10204081632654 487.8775510204081,135.76530612244895 487.5714285714285,136.42857142857142 487.265306122449,137.09183673469386 486.95918367346934,137.7551020408163 486.6530612244898,138.41836734693877 486.3469387755102,139.0816326530612 486.0408163265306,139.74489795918365 485.734693877551,140.40816326530611 485.4285714285715,141.07142857142856 485.1224489795918,141.73469387755102 484.8163265306123,142.39795918367346 484.5102040816327,143.06122448979593 484.204081632653,143.72448979591834 483.8979591836735,144.38775510204079 483.59183673469386,145.05102040816325 483.28571428571433,145.71428571428572 482.9795918367347,146.37755102040816 482.6734693877551,147.0408163265306 482.36734693877554,147.70408163265304 482.0612244897959,148.36734693877548 481.7551020408164,149.03061224489795 481.44897959183675,149.69387755102042 481.14285714285717,150.35714285714286 480.83673469387753,151.0204081632653 480.53061224489795,151.68367346938774 480.2244897959184,152.3469387755102 479.9183673469388,153.01020408163265 479.61224489795916,153.6734693877551 479.3061224489796,154.33673469387753 479.0,155.0" stroke="#e4815e" stroke-width="1" /> 0 <polyline fill="none" points="585.0,90.0 584.0714285714286,90.66326530612244 583.142857142857,91.32653061224487 582.2142857142858,91.98979591836735 581.2857142857143,9

**Lets set custom labels instead of default ones by using  `.setCustomLabels()` parameter**

In [0]:
reModel = RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setCustomLabels({"1": "is_related", "0": "not_related"})

ade_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_model = ade_pipeline.fit(empty_data)

re_ade_clinical download started this may take some time.
[OK!]


In [0]:
ade_lmodel = LightPipeline(ade_model)

text ="""A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

ade_results = ade_lmodel.fullAnnotate(text)

In [0]:
print(text)

rel_df = get_relations_df (ade_results)

rel_df


A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.


,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,25,32,naproxen,DRUG,137,148,tense bullae,ADE,is_related,1.0
1,0,25,32,naproxen,DRUG,154,210,cutaneous fragility on the face and the back of the hands,ADE,is_related,0.99999976
2,0,87,95,oxaprozin,DRUG,137,148,tense bullae,ADE,is_related,1.0
3,0,87,95,oxaprozin,DRUG,154,210,cutaneous fragility on the face and the back of the hands,ADE,is_related,1.0


In [0]:
visualizer = RelationExtractionVisualizer()
vis = visualizer.display(ade_results[0], 'relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

A 44-year-old man taking naproxen DRUG for chronic low back pain and a 20-year-old woman on oxaprozin DRUG for rheumatoid arthritis presented with tense bullae ADE and cutaneous fragility on the face and the back of the hands ADE . <polyline fill="none" points="291.0,90.0 291.07142857142856,91.68367346938774 291.1428571428571,93.3673469387755 291.2142857142857,95.05102040816327 291.2857142857143,96.73469387755102 291.35714285714283,98.41836734693878 291.42857142857144,100.10204081632652 291.5000000000001,101.7857142857143 291.57142857142856,103.46938775510205 291.64285714285717,105.15306122448982 291.71428571428567,106.83673469387755 291.7857142857143,108.52040816326529 291.85714285714283,110.20408163265306 291.92857142857144,111.88775510204081 292.00000000000006,113.57142857142857 292.0714285714286,115.25510204081633 292.1428571428572,116.9387755102041 292.2142857142858,118.62244897959187 292.2857142857143,120.3061224489796 292.35714285714283,121.98979591836735 292.4285714285714,123.67346938775508 292.5,125.35714285714286 292.5714285714286,127.0408163265306 292.6428571428571,128.72448979591837 292.7142857142858,130.40816326530611 292.78571428571433,132.09183673469389 292.85714285714283,133.77551020408163 292.92857142857144,135.45918367346937 293.00000000000006,137.14285714285717 293.0714285714286,138.8265306122449 293.14285714285717,140.51020408163265 293.2142857142857,142.1938775510204 293.28571428571433,143.87755102040813 293.35714285714283,145.5612244897959 293.42857142857144,147.24489795918367 293.5,148.92857142857142 293.5714285714286,150.6122448979592 293.64285714285717,152.29591836734693 293.7142857142857,153.9795918367347 293.78571428571433,155.66326530612244 293.8571428571429,157.3469387755102 293.92857142857144,159.03061224489795 294.0,160.71428571428572 294.07142857142856,162.39795918367346 294.1428571428571,164.0816326530612 294.2142857142857,165.76530612244898 294.28571428571433,167.44897959183675 294.3571428571429,169.1326530612245 294.42857142857144,170.81632653061223 294.5,172.5 294.5,172.5 294.57142857142856,174.18367346938774 294.6428571428571,175.86734693877548 294.71428571428567,177.55102040816328 294.7857142857143,179.23469387755102 294.85714285714283,180.91836734693877 294.9285714285714,182.6020408163265 295.00000000000006,184.2857142857143 295.07142857142856,185.96938775510208 295.14285714285717,187.65306122448985 295.2142857142857,189.33673469387753 295.2857142857142,191.0204081632653 295.3571428571428,192.70408163265304 295.4285714285714,194.3877551020408 295.49999999999994,196.07142857142856 295.57142857142856,197.75510204081635 295.6428571428571,199.4387755102041 295.7142857142858,201.12244897959184 295.7857142857143,202.80612244897958 295.85714285714283,204.48979591836735 295.9285714285714,206.17346938775506 295.99999999999994,207.85714285714283 296.07142857142856,209.5408163265306 296.1428571428571,211.22448979591834 296.2142857142857,212.90816326530614 296.2857142857142,214.59183673469389 296.3571428571428,216.27551020408163 296.4285714285714,217.9591836734694 296.5,219.64285714285714 296.57142857142856,221.3265306122449 296.6428571428571,223.01020408163265 296.7142857142857,224.69387755102042 296.7857142857142,226.37755102040813 296.8571428571429,228.0612244897959 296.9285714285714,229.74489795918367 297.0,231.42857142857144 297.07142857142856,233.1122448979592 297.1428571428571,234.79591836734693 297.2142857142857,236.4795918367347 297.2857142857143,238.16326530612244 297.3571428571429,239.84693877551018 297.42857142857144,241.53061224489795 297.5,243.21428571428572 297.57142857142856,244.89795918367344 297.6428571428571,246.5816326530612 297.7142857142857,248.26530612244898 297.7857142857143,249.94897959183675 297.8571428571429,251.63265306122452 297.9285714285714,253.31632653061223 298.0,255.0" stroke="#72445C" stroke-width="1" /> is_related <polyline fill="none" points="78.0,190.0 80.09183673469387,190.66326530612247 82.18367346938774,191.32653061224485 84.27551020408164,191.98979591836735 8

## Clinical RE

**The set of relations defined in the 2010 i2b2 relation challenge**

TrIP: A certain treatment has improved or cured a medical problem (eg, ‘infection resolved with antibiotic course’)

TrWP: A patient's medical problem has deteriorated or worsened because of or in spite of a treatment being administered (eg, ‘the tumor was growing despite the drain’)

TrCP: A treatment caused a medical problem (eg, ‘penicillin causes a rash’)

TrAP: A treatment administered for a medical problem (eg, ‘Dexamphetamine for narcolepsy’)

TrNAP: The administration of a treatment was avoided because of a medical problem (eg, ‘Ralafen which is contra-indicated because of ulcers’)

TeRP: A test has revealed some medical problem (eg, ‘an echocardiogram revealed a pericardial effusion’)

TeCP: A test was performed to investigate a medical problem (eg, ‘chest x-ray done to rule out pneumonia’)

PIP: Two problems are related to each other (eg, ‘Azotemia presumed secondary to sepsis’)

In [0]:
clinical_ner_tagger = MedicalNerModel()\
    .pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_Model = RelationExtractionModel()\
    .pretrained("re_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
    .setRelationPairs(["problem-test", "problem-treatment"]) # we can set the possible relation pairs (if not set, all the relations will be calculated)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    clinical_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])


empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_clinical download started this may take some time.
[OK!]
re_clinical download started this may take some time.
[OK!]


In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df[rel_df.relation!="O"]


,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
2,0,322,324,BMI,TEST,381,388,polyuria,PROBLEM,TeRP,0.99756753
4,0,322,324,BMI,TEST,405,417,poor appetite,PROBLEM,TeRP,0.9980003
5,0,322,324,BMI,TEST,425,432,vomiting,PROBLEM,TrAP,0.894692
6,1,512,522,amoxicillin,TREATMENT,528,556,a respiratory tract infection,PROBLEM,TrAP,0.9999393
7,2,571,579,metformin,TREATMENT,617,620,T2DM,PROBLEM,TrAP,0.99999905
8,2,583,591,glipizide,TREATMENT,617,620,T2DM,PROBLEM,TrAP,0.9999999
9,2,599,611,dapagliflozin,TREATMENT,617,620,T2DM,PROBLEM,TrAP,0.99999976
10,2,599,611,dapagliflozin,TREATMENT,659,661,HTG,PROBLEM,TrAP,0.9998598
13,2,626,637,atorvastatin,TREATMENT,659,661,HTG,PROBLEM,TrAP,0.99999547
14,2,643,653,gemfibrozil,TREATMENT,659,661,HTG,PROBLEM,TrAP,1.0


The RelationExtractionModel annotator is now equipped with the `setMultiClass()` method, which provides the option to specify whether the model should return only the label with the highest confidence score or include all labels in its output.



In [0]:
reModel = RelationExtractionModel.pretrained("re_clinical", "en", "clinical/models")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setCustomLabels({"1": "is_related", "0": "not_related"})\
    .setMultiClass(True) # or Default value is False\


ade_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer,
    words_embedder,
    pos_tagger,
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_model2 = ade_pipeline.fit(empty_data)

re_clinical download started this may take some time.
[OK!]


In [0]:
text ="""A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

lp = LightPipeline(ade_model2)

res = lp.fullAnnotate(text)

In [0]:
for rel in res[0]['relations']:
  print(rel)

# after .setMultiClass(True), in metadata, you can observe confidences associated with various other classes of relationships.

Annotation(category, 25, 148, O, {'TrWP_confidence': '3.433041E-33', 'chunk2': 'tense bullae', 'TrIP_confidence': '1.006051E-34', 'TrAP_confidence': '1.6303902E-22', 'TrNAP_confidence': '6.1178304E-36', 'confidence': '1.0', 'entity2_end': '148', 'chunk1': 'naproxen', 'entity1': 'DRUG', 'entity2_begin': '137', 'TeRP_confidence': '1.6885795E-15', 'TeCP_confidence': '8.3169976E-30', 'O_confidence': '1.0', 'chunk2_confidence': '0.77250004', 'entity1_begin': '25', 'sentence': '0', 'PIP_confidence': '0.0', 'direction': 'both', 'TrCP_confidence': '3.1979797E-34', 'entity1_end': '32', 'entity2': 'ADE', 'chunk1_confidence': '0.996'}, [])
Annotation(category, 25, 210, TeRP, {'TrWP_confidence': '1.4348979E-7', 'chunk2': 'cutaneous fragility on the face and the back of the hands', 'TrIP_confidence': '3.372166E-37', 'TrAP_confidence': '2.045877E-10', 'TrNAP_confidence': '5.6984136E-20', 'confidence': '0.97853744', 'entity2_end': '210', 'chunk1': 'naproxen', 'entity1': 'DRUG', 'entity2_begin': '154'

## Clinical Temporal Events RE

Temporal relations, or temporal links (denoted by the TLINK tag), indicate whether and how two EVENTs, two TIME, or an EVENT and a TIME related to each other in the clinical timeline. There are 3 type of relations here and below are some examples of Relations, with square brackets indicating EVENT and TIME connected by a temporal link:

**`BEFORE`**

The patient was given stress dose steroids prior to his surgery. ([stress dose steroids] `BEFORE` [his surgery])

The patient had an undocumented history of possible atrial fibrillation prior to admission. ([possible atrial fibrillation] `BEFORE` [admission])

His nasogastric tube was discontinued on 05-26-98. ([His nasogastric] `BEFORE` [05-26-98])

**`AFTER`**

Before admission, he had another serious concussion. ([admission] `AFTER` [another serious concussion])

On postoperative day No 1, he was started on Percocet. ([Percocet] `AFTER` [postoperative day No 1])


**`OVERLAP`**

She denies any fevers or chills. ([fevers] `OVERLAP` [chills])

The patient's serum creatinine on discharge date, 2012-05-06, was 1.9. ([discharge date] `OVERLAP` [2012-05-06])

His preoperative workup was completed and included a normal white count ([a normal white count] `OVERLAP` [His preoperative workup])

The patient had an undocumented history of possible atrial fibrillation prior to admission. ([possible atrial fibrillation] `OVERLAP` [admission])


| Relation | Recall | Precision | F1 |
| --- | --- | --- | --- |
| OVERLAP | 0.81 | 0.73 | **0.77** |
| BEFORE | 0.85 | 0.88 | **0.86** |
| AFTER | 0.38 | 0.46 | **0.43** |

This RE model works with `ner_events_clinical` NER model and expect the following entities as inputs:

[`OCCURRENCE`,
 `DATE`,
 `DURATION`,
 `EVIDENTIAL`,
 `TEST`,
 `PROBLEM`,
 `TREATMENT`,
 `CLINICAL_DEPT`,
 `FREQUENCY`,
 `TIME`]

In [0]:
events_ner_tagger = MedicalNerModel()\
    .pretrained("ner_events_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_Model = RelationExtractionModel()\
    .pretrained("re_temporal_events_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
    .setPredictionThreshold(0.9)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_events_clinical download started this may take some time.
[OK!]
re_temporal_events_clinical download started this may take some time.
[OK!]


In [0]:
events_ner_tagger.getClasses()

['O',
 'B-OCCURRENCE',
 'B-DATE',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-DURATION',
 'I-DURATION',
 'B-EVIDENTIAL',
 'I-OCCURRENCE',
 'B-TREATMENT',
 'I-TREATMENT',
 'I-DATE',
 'B-TEST',
 'I-TEST',
 'B-CLINICAL_DEPT',
 'I-CLINICAL_DEPT',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'I-EVIDENTIAL',
 'B-TIME',
 'I-TIME']

In [0]:
text ="She had a retinal degeneration after the accident in 2005 May"

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,8,29,a retinal degeneration,PROBLEM,37,48,the accident,OCCURRENCE,AFTER,1.0
1,0,8,29,a retinal degeneration,PROBLEM,53,60,2005 May,DATE,AFTER,0.9616042
2,0,37,48,the accident,OCCURRENCE,53,60,2005 May,DATE,OVERLAP,1.0


In [0]:
text ="On 9–28-92, the patient will return for chemotherapy and she will follow up with her primary doctor, for PT and Coumadin dosing on Monday."

annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df[(rel_df.relation!="O")&(rel_df.entity1!=rel_df.entity2)]


,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,3,9,9–28-92,DATE,40,51,chemotherapy,TREATMENT,OVERLAP,0.999999
1,0,40,51,chemotherapy,TREATMENT,66,74,follow up,OCCURRENCE,AFTER,0.995435
2,0,40,51,chemotherapy,TREATMENT,81,98,her primary doctor,CLINICAL_DEPT,AFTER,0.967238
4,0,66,74,follow up,OCCURRENCE,81,98,her primary doctor,CLINICAL_DEPT,OVERLAP,0.983266
5,0,66,74,follow up,OCCURRENCE,105,106,PT,TREATMENT,OVERLAP,0.986475
6,0,66,74,follow up,OCCURRENCE,131,136,Monday,DATE,AFTER,0.968594
7,0,81,98,her primary doctor,CLINICAL_DEPT,131,136,Monday,DATE,AFTER,0.999235
9,0,105,106,PT,TREATMENT,131,136,Monday,DATE,AFTER,0.982908


**Admission-Discharge Events Exraction**

We can also extract `ADMISSION` and `DISCHARGE` entities in addition to `ner_events_clinical` model entities using `ner_events_admission_clinical` NER model and expect the following entities as inputs:
   
[`EVIDENTIAL`,
 `OCCURRENCE`,
 `TREATMENT`,
 `DATE`,
 `ADMISSION`,
 `TIME`,
 `FREQUENCY`,
 `CLINICAL_DEPT`,
 `DURATION`,
 `PROBLEM`,
 `TEST`,
 `DISCHARGE`]

In [0]:
events_admission_ner_tagger = MedicalNerModel()\
    .pretrained("ner_events_admission_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_admission_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_events_admission_clinical download started this may take some time.
[OK!]


In [0]:
events_admission_ner_tagger.getClasses()

['O',
 'B-EVIDENTIAL',
 'I-OCCURRENCE',
 'I-TREATMENT',
 'I-DATE',
 'B-ADMISSION',
 'B-TREATMENT',
 'I-TIME',
 'I-FREQUENCY',
 'B-FREQUENCY',
 'B-OCCURRENCE',
 'I-CLINICAL_DEPT',
 'B-DURATION',
 'I-PROBLEM',
 'I-TEST',
 'B-DATE',
 'B-TIME',
 'I-EVIDENTIAL',
 'I-DURATION',
 'B-DISCHARGE',
 'B-TEST',
 'B-PROBLEM',
 'B-CLINICAL_DEPT']

In [0]:
text ="""She is admitted to The John Hopkins Hospital on Monday with a history of gestational diabetes mellitus diagnosed.  
She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 
12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge. 
"""

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df[(rel_df.relation!="O")]


,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,7,14,admitted,ADMISSION,19,43,The John Hopkins Hospital,CLINICAL_DEPT,AFTER,0.982727
1,0,7,14,admitted,ADMISSION,48,53,Monday,DATE,OVERLAP,1.000000
2,0,19,43,The John Hopkins Hospital,CLINICAL_DEPT,48,53,Monday,DATE,OVERLAP,1.000000
3,0,48,53,Monday,DATE,73,101,gestational diabetes mellitus,PROBLEM,BEFORE,0.988257
4,1,132,156,the endocrinology service,CLINICAL_DEPT,184,193,03/02/2018,DATE,OVERLAP,0.987573
5,1,170,179,discharged,DISCHARGE,184,193,03/02/2018,DATE,OVERLAP,1.000000
6,1,170,179,discharged,DISCHARGE,210,225,insulin glargine,TREATMENT,AFTER,0.973975
7,1,170,179,discharged,DISCHARGE,241,254,insulin lispro,TREATMENT,AFTER,0.941394
8,1,184,193,03/02/2018,DATE,241,254,insulin lispro,TREATMENT,OVERLAP,0.980152
9,1,261,269,metformin,TREATMENT,279,293,two times a day,FREQUENCY,OVERLAP,1.000000


**Test-Result-Date Extraction**

There is another Relation Extraction model to link clinical tests to test results and dates to clinical entities: `re_test_result_date`

In [0]:
trd_ner_tagger = MedicalNerModel()\
    .pretrained("jsl_ner_wip_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")       

trd_re_Model = RelationExtractionModel()\
    .pretrained("re_test_result_date", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    trd_ner_tagger,
    ner_chunker,
    dependency_parser,
    trd_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

jsl_ner_wip_clinical download started this may take some time.
[OK!]
re_test_result_date download started this may take some time.
[OK!]


In [0]:
text = 'Hospitalized with pneumonia on 11 June 2019, confirmed by a positive PCR of any specimen, evidenced by SPO2 </= 93%.'

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df[(rel_df.relation!="O")]

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,18,26,pneumonia,Disease_Syndrome_Disorder,31,42,11 June 2019,Date,is_date_of,1.000000
2,0,18,26,pneumonia,Disease_Syndrome_Disorder,69,71,PCR,Test,is_finding_of,0.969077
3,0,18,26,pneumonia,Disease_Syndrome_Disorder,103,114,SPO2 </= 93%,O2_Saturation,is_finding_of,0.999999
4,0,31,42,11 June 2019,Date,60,67,positive,Test_Result,is_date_of,0.969872
7,0,60,67,positive,Test_Result,69,71,PCR,Test,is_result_of,1.000000
8,0,60,67,positive,Test_Result,103,114,SPO2 </= 93%,O2_Saturation,is_result_of,0.993989
9,0,69,71,PCR,Test,103,114,SPO2 </= 93%,O2_Saturation,is_result_of,0.511514


## Human Phenotype - Gene RE

https://github.com/lasigeBioTM/PGR

Human phenotype-gene relations are fundamental to fully understand the origin of some phenotypic abnormalities and their associated diseases. Biomedical literature is the most comprehensive source of these relations, however, we need Relation Extraction tools to automatically recognize them. We present the Phenotype-Gene Relations (PGR) model, trained on a silver standard corpus of human phenotype and gene annotations and their relations. 

It extracts 2 label: `True` or `False`

In [0]:
pgr_ner_tagger = MedicalNerModel()\
    .pretrained("ner_human_phenotype_gene_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

pgr_re_Model = RelationExtractionModel()\
    .pretrained("re_human_phenotype_gene_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setRelationPairs(["hp-gene",'gene-hp'])\
    .setMaxSyntacticDistance(4)\

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    pgr_ner_tagger,
    ner_chunker,
    dependency_parser,
    pgr_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_human_phenotype_gene_clinical download started this may take some time.
[OK!]
re_human_phenotype_gene_clinical download started this may take some time.
[OK!]


In [0]:
text = "She has a retinal degeneration, hearing loss and renal failure, short stature, \
Mutations in the SH3PXD2B gene coding for the Tks4 protein are responsible for the autosomal recessive. "

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!=0)]

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,32,43,hearing loss,HP,96,103,SH3PXD2B,GENE,1,0.9999999
1,0,49,61,renal failure,HP,96,103,SH3PXD2B,GENE,1,0.99992704
2,0,64,76,short stature,HP,96,103,SH3PXD2B,GENE,1,0.9999391


## Drug-Drug Interaction RE

In clinical application, two or more drugs are often used in combination to achieve conducive results, such as synergistic effect, increasing therapeutic effect and reducing or delaying the occurrence of
drug resistance. However, there is a potential for harmful drug-drug
interactions (DDIs) to occur when two or more drugs are taken at the
same time or at certain interval, which can reduce or invalidate the
efficacy of drugs, and increase toxicity or even cause death. Therefore,
in order to prevent harmful drug-drug interaction (DDI), medical staff
often spend much time in reviewing the relevant drug alert literature
and drug knowledge bases. 

**ref**: *Drug-drug interaction extraction via hybrid neural networks on biomedical literature
https://www-sciencedirect-com.ezproxy.leidenuniv.nl:2443/science/article/pii/S1532046420300605*

In [0]:
ner_tagger = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_tags"]) \
    .setOutputCol("ner_chunk")

ddi_re_model = RelationExtractionModel()\
    .pretrained("re_drug_drug_interaction_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setRelationPairs(["drug-drug"])\
    .setMaxSyntacticDistance(4)\

ddi_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_converter,
    dependency_parser,
    ddi_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ddi_model = ddi_pipeline.fit(empty_data)

ner_posology download started this may take some time.
[OK!]
re_drug_drug_interaction_clinical download started this may take some time.
[OK!]


In [0]:
text='When carbamazepine is withdrawn from the combination therapy, aripiprazole dose should then be reduced. \
If additional adrenergic drugs are to be administered by any route, \
they should be used with caution because the pharmacologically predictable sympathetic effects of Metformin may be potentiated'

lmodel = LightPipeline(ddi_model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations, chunk_col="ner_chunk")

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,5,17,carbamazepine,DRUG,62,73,aripiprazole,DRUG,DDI-advise,0.99238014


In [0]:
annotations[0]['ner_chunk']

[Annotation(chunk, 5, 17, carbamazepine, {'chunk': '0', 'confidence': '0.9968', 'ner_source': 'ner_chunk', 'entity': 'DRUG', 'sentence': '0'}, []),
 Annotation(chunk, 62, 73, aripiprazole, {'chunk': '1', 'confidence': '0.9998', 'ner_source': 'ner_chunk', 'entity': 'DRUG', 'sentence': '0'}, []),
 Annotation(chunk, 118, 133, adrenergic drugs, {'chunk': '2', 'confidence': '0.7984', 'ner_source': 'ner_chunk', 'entity': 'DRUG', 'sentence': '1'}, []),
 Annotation(chunk, 270, 278, Metformin, {'chunk': '3', 'confidence': '0.9998', 'ner_source': 'ner_chunk', 'entity': 'DRUG', 'sentence': '1'}, [])]

## Drug-Protein Interaction RE

Detect interactions between chemical compounds/drugs and genes/proteins using Spark NLP’s `RelationExtractionModel()` by classifying whether a specified semantic relation holds between a chemical and gene entities within a sentence or document. <br/>

The entity labels used during training were derived from the custom NER model created by our team for the **DrugProt** corpus. These include `CHEMICAL` for chemical compounds/drugs, `GENE` for genes/proteins and `GENE_AND_CHEMICAL` for entity mentions of type `GENE` and of type `CHEMICAL` that overlap (such as enzymes and small peptides). <br/>

The relation categories from the **DrugProt** corpus were condensed from 13 categories to 10 categories due to low numbers of examples for certain categories. This merging process involved grouping the `SUBSTRATE_PRODUCT-OF` and `SUBSTRATE` relation categories together and grouping the `AGONIST-ACTIVATOR`, `AGONIST-INHIBITOR` and `AGONIST` relation categories together.

**ref**: *DrugProt corpus: Biocreative VII Track 1 - Text mining drug and chemical-protein interactionshttps://zenodo.org/record/5119892*

In [0]:
drugprot_ner_tagger = MedicalNerModel.pretrained("ner_drugprot_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

drugprot_re_model = RelationExtractionModel()\
        .pretrained("re_drugprot_clinical", "en", 'clinical/models')\
        .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
        .setOutputCol("relations")\
        .setMaxSyntacticDistance(4)\
        .setPredictionThreshold(0.5)\
        .setRelationPairs(["checmical-gene", "chemical-gene_and_chemical", "gene_and_chemical-gene"]) 

pipeline = Pipeline(stages=[
        documenter, 
        sentencer,
        tokenizer,
        words_embedder, 
        drugprot_ner_tagger,
        ner_converter, 
        pos_tagger, 
        dependency_parser, 
        drugprot_re_model])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_drugprot_clinical download started this may take some time.
[OK!]
re_drugprot_clinical download started this may take some time.
[OK!]


In [0]:
text='''Lipid specific activation of the murine P4-ATPase Atp8a1 (ATPase II). The asymmetric transbilayer distribution of phosphatidylserine (PS) in the mammalian plasma membrane and secretory vesicles is maintained, in part, by an ATP-dependent transporter. This aminophospholipid "flippase" selectively transports PS to the cytosolic leaflet of the bilayer and is sensitive to vanadate, Ca(2+), and modification by sulfhydryl reagents. Although the flippase has not been positively identified, a subfamily of P-type ATPases has been proposed to function as transporters of amphipaths, including PS and other phospholipids. A candidate PS flippase ATP8A1 (ATPase II), originally isolated from bovine secretory vesicles, is a member of this subfamily based on sequence homology to the founding member of the subfamily, the yeast protein Drs2, which has been linked to ribosomal assembly, the formation of Golgi-coated vesicles, and the maintenance of PS asymmetry. To determine if ATP8A1 has biochemical characteristics consistent with a PS flippase, a murine homologue of this enzyme was expressed in insect cells and purified. The purified Atp8a1 is inactive in detergent micelles or in micelles containing phosphatidylcholine, phosphatidic acid, or phosphatidylinositol, is minimally activated by phosphatidylglycerol or phosphatidylethanolamine (PE), and is maximally activated by PS. The selectivity for PS is dependent upon multiple elements of the lipid structure. Similar to the plasma membrane PS transporter, Atp8a1 is activated only by the naturally occurring sn-1,2-glycerol isomer of PS and not the sn-2,3-glycerol stereoisomer. Both flippase and Atp8a1 activities are insensitive to the stereochemistry of the serine headgroup. Most modifications of the PS headgroup structure decrease recognition by the plasma membrane PS flippase. Activation of Atp8a1 is also reduced by these modifications; phosphatidylserine-O-methyl ester, lysophosphatidylserine, glycerophosphoserine, and phosphoserine, which are not transported by the plasma membrane flippase, do not activate Atp8a1. Weakly translocated lipids (PE, phosphatidylhydroxypropionate, and phosphatidylhomoserine) are also weak Atp8a1 activators. However, N-methyl-phosphatidylserine, which is transported by the plasma membrane flippase at a rate equivalent to PS, is incapable of activating Atp8a1 activity. These results indicate that the ATPase activity of the secretory granule Atp8a1 is activated by phospholipids binding to a specific site whose properties (PS selectivity, dependence upon glycerol but not serine, stereochemistry, and vanadate sensitivity) are similar to, but distinct from, the properties of the substrate binding site of the plasma membrane flippase.'''

lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [0]:
results[0].keys()

dict_keys(['sentences', 'document', 'ner_chunks', 'ner_tags', 'relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [0]:
results[0]["ner_chunks"]

[Annotation(chunk, 33, 48, murine P4-ATPase, {'chunk': '0', 'confidence': '0.75505', 'ner_source': 'ner_chunks', 'entity': 'GENE', 'sentence': '0'}, []),
 Annotation(chunk, 50, 55, Atp8a1, {'chunk': '1', 'confidence': '0.9911', 'ner_source': 'ner_chunks', 'entity': 'GENE', 'sentence': '0'}, []),
 Annotation(chunk, 58, 66, ATPase II, {'chunk': '2', 'confidence': '0.8749', 'ner_source': 'ner_chunks', 'entity': 'GENE', 'sentence': '0'}, []),
 Annotation(chunk, 114, 131, phosphatidylserine, {'chunk': '3', 'confidence': '0.9794', 'ner_source': 'ner_chunks', 'entity': 'CHEMICAL', 'sentence': '1'}, []),
 Annotation(chunk, 134, 135, PS, {'chunk': '4', 'confidence': '0.7845', 'ner_source': 'ner_chunks', 'entity': 'CHEMICAL', 'sentence': '1'}, []),
 Annotation(chunk, 224, 248, ATP-dependent transporter, {'chunk': '5', 'confidence': '0.77744997', 'ner_source': 'ner_chunks', 'entity': 'GENE_AND_CHEMICAL', 'sentence': '1'}, []),
 Annotation(chunk, 256, 282, aminophospholipid "flippase, {'chunk': '6

In [0]:
results[0]["relations"]

[Annotation(category, 114, 248, ACTIVATOR, {'chunk2': 'ATP-dependent transporter', 'confidence': '0.988005', 'entity2_end': '248', 'chunk1': 'phosphatidylserine', 'entity1': 'CHEMICAL', 'entity2_begin': '224', 'chunk2_confidence': '0.77744997', 'entity1_begin': '114', 'sentence': '1', 'direction': 'both', 'entity1_end': '131', 'entity2': 'GENE_AND_CHEMICAL', 'chunk1_confidence': '0.9794'}, []),
 Annotation(category, 134, 248, ACTIVATOR, {'chunk2': 'ATP-dependent transporter', 'confidence': '0.579751', 'entity2_end': '248', 'chunk1': 'PS', 'entity1': 'CHEMICAL', 'entity2_begin': '224', 'chunk2_confidence': '0.77744997', 'entity1_begin': '134', 'sentence': '1', 'direction': 'both', 'entity1_end': '135', 'entity2': 'GENE_AND_CHEMICAL', 'chunk1_confidence': '0.7845'}, []),
 Annotation(category, 256, 309, SUBSTRATE, {'chunk2': 'PS', 'confidence': '0.9999436', 'entity2_end': '309', 'chunk1': 'aminophospholipid "flippase', 'entity1': 'GENE_AND_CHEMICAL', 'entity2_begin': '308', 'chunk2_confid

In [0]:
rel_df = get_relations_df (results)

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,1,114,131,phosphatidylserine,CHEMICAL,224,248,ATP-dependent transporter,GENE_AND_CHEMICAL,ACTIVATOR,0.988005
1,1,134,135,PS,CHEMICAL,224,248,ATP-dependent transporter,GENE_AND_CHEMICAL,ACTIVATOR,0.579751
2,2,256,282,"aminophospholipid ""flippase",GENE_AND_CHEMICAL,308,309,PS,CHEMICAL,SUBSTRATE,0.9999436
3,4,629,639,PS flippase,GENE_AND_CHEMICAL,641,646,ATP8A1,GENE,SUBSTRATE,0.90787965
4,5,973,978,ATP8A1,GENE,1030,1040,PS flippase,GENE_AND_CHEMICAL,SUBSTRATE,0.5078753


## Chemical–Protein Interactions (ChemProt RE)

Accurately detecting the interactions between chemicals and proteins is a crucial task that plays a key role in precision medicine, drug discovery and basic clinical research. Currently, PubMed contains >28 million articles, and its annual growth rate is more than a million articles each year. A large amount of valuable chemical–protein interactions (CPIs) are hidden in the biomedical literature. There is an increasing interest in CPI extraction from the biomedical literature.

Since manually extracting biomedical relations such as protein–protein interactions (PPI) and drug–drug interactions (DDI) is costly and time-consuming, some computational methods have been successfully proposed for automatic biomedical relation extraction.

To date, most studies on the biomedical relation extraction have focused on the PPIs and DDIs, but a few attempts have been made to extract CPIs. The BioCreative VI ChemProt shared task released the ChemProt dataset for CPI extraction, which is the first challenge for extracting CPIs.

Computational CPI extraction is generally approached as a task of classifying whether a specified semantic relation holds between the chemical and protein entities within a sentence or document. The ChemProt corpus is a manually annotated CPI dataset, which greatly promotes the development of CPI extraction approaches. 

ref: https://academic.oup.com/database/article/doi/10.1093/database/baz054/5498050

| Relation | Recall | Precision | F1 | F1 (Zhang, Yijia, et al., 2019) |
| --- | --- | --- | --- | --- |
| CPR:3 | 0.47 | 0.59 | **0.52** | 0.594 |
| CPR:4 | 0.72 | 0.81 | **0.77** | 0.718 |
| CPR:5 | 0.43 | 0.88 | **0.58** | 0.657 |
| CPR:6 | 0.59 | 0.89 | **0.71** | 0.725 |
| CPR:9 | 0.62 | 0.84 | **0.71** | 0.501 |
|avg. |  |  | **0.66** | 0.64 |

Here are the relation types

| Group | Eval. | CHEMPROT relations belonging to this group |
|-------|-------|--------------------------------------------|
| CPR:1 | N     | PART_OF                                    |
| CPR:2  	| N     	| REGULATOR\|DIRECT REGULATOR\|INDIRECT REGULATOR   	|
| CPR:3  	| Y     	| UPREGULATOR ACTIVATOR INDIRECT UPREGULATOR        	|
| CPR:4  	| Y     	| DOWNREGULATORIINHIBITORIINDIRECTDOWNREGULATOR     	|
| CPR:5  	| Y     	| AGONISTIAGONIST-ACTIVATOR\|AGONIST-INHIBITOR      	|
| CPR:6  	| Y     	| ANTAGONIST                                        	|
| CPR:7  	| N     	| MODULATORIMODULATOR-ACTIVATORIMODULATOR-INHIBITOR 	|
| CPR:8  	| N     	| COFACTOR                                          	|
| CPR:9  	| Y     	| SUBSTRATEIPRODUCT OF\|SUBSTRATE PRODUCT OF        	|
| CPR:10 	| N     	| NOT                                               	|

ChemProt RE works well with `ner_chemprot_clinical` find relationships between the following entities

`CHEMICAL`: Chemical entity mention type; 

`GENE-Y`: gene/protein mention type that can be normalized or associated to a biological database identifier; 

`GENE-N`: gene/protein mention type that cannot be normalized to a database identifier.

In [0]:
ner_tagger = MedicalNerModel()\
    .pretrained("ner_chemprot_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_tags"]) \
    .setOutputCol("ner_chunk")

chemprot_re_model = RelationExtractionModel()\
    .pretrained("re_chemprot_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\

chemprot_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_converter,
    dependency_parser,
    chemprot_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chemprot_model = chemprot_pipeline.fit(empty_data)

ner_chemprot_clinical download started this may take some time.
[OK!]
re_chemprot_clinical download started this may take some time.
[OK!]


In [0]:
text='''
In this study, we examined the effects of mitiglinide on various cloned K(ATP) channels (Kir6.2/SUR1, Kir6.2/SUR2A, and Kir6.2/SUR2B) reconstituted in COS-1 cells, and compared them to another meglitinide-related compound, nateglinide. Patch-clamp analysis using inside-out recording configuration showed that mitiglinide inhibits the Kir6.2/SUR1 channel currents in a dose-dependent manner (IC50 value, 100 nM) but does not significantly inhibit either Kir6.2/SUR2A or Kir6.2/SUR2B channel currents even at high doses (more than 10 microM). Nateglinide inhibits Kir6.2/SUR1 and Kir6.2/SUR2B channels at 100 nM, and inhibits Kir6.2/SUR2A channels at high concentrations (1 microM). Binding experiments on mitiglinide, nateglinide, and repaglinide to SUR1 expressed in COS-1 cells revealed that they inhibit the binding of [3H]glibenclamide to SUR1 (IC50 values: mitiglinide, 280 nM; nateglinide, 8 microM; repaglinide, 1.6 microM), suggesting that they all share a glibenclamide binding site. The insulin responses to glucose, mitiglinide, tolbutamide, and glibenclamide in MIN6 cells after chronic mitiglinide, nateglinide, or repaglinide treatment were comparable to those after chronic tolbutamide and glibenclamide treatment. These results indicate that, similar to the sulfonylureas, mitiglinide is highly specific to the Kir6.2/SUR1 complex, i.e., the pancreatic beta-cell K(ATP) channel, and suggest that mitiglinide may be a clinically useful anti-diabetic drug.
'''
lmodel = LightPipeline(chemprot_model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations, chunk_col="ner_chunk")

rel_df[rel_df.entity1!=rel_df.entity2]

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,43,53,mitiglinide,CHEMICAL,80,87,channels,GENE-N,CPR:2,0.53757155
1,0,80,87,channels,GENE-N,224,234,nateglinide,CHEMICAL,CPR:2,0.99001455
4,3,706,716,mitiglinide,CHEMICAL,751,754,SUR1,GENE-Y,CPR:4,0.629676
6,3,719,729,nateglinide,CHEMICAL,751,754,SUR1,GENE-Y,CPR:2,0.999816
7,3,736,746,repaglinide,CHEMICAL,751,754,SUR1,GENE-Y,CPR:2,0.96780753
8,3,823,839,[3H]glibenclamide,CHEMICAL,844,847,SUR1,GENE-Y,CPR:2,1.0
11,6,998,1004,insulin,GENE-N,1019,1025,glucose,CHEMICAL,CPR:2,0.63596284
12,6,998,1004,insulin,GENE-N,1028,1038,mitiglinide,CHEMICAL,CPR:2,0.5456071
13,6,998,1004,insulin,GENE-N,1041,1051,tolbutamide,CHEMICAL,CPR:3,0.9872542
14,6,998,1004,insulin,GENE-N,1058,1070,glibenclamide,CHEMICAL,CPR:3,0.98967665


# Train a Custom Relation Extraction Model

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_clinical_rel_dataset.csv

dbutils.fs.cp("file:/databricks/driver/i2b2_clinical_rel_dataset.csv", "dbfs:/")

True

In [0]:
data = spark.read.option("header","true").format("csv").load("/i2b2_clinical_rel_dataset.csv")

data = data.select( 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel','dataset')

data.show(10)

# you only need these columns>> 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel'
# ('dataset' column is optional)

+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
|            sentence|firstCharEnt1|firstCharEnt2|lastCharEnt1|lastCharEnt2|              chunk1|              chunk2|   label1|   label2|  rel|dataset|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
|VITAL SIGNS - Tem...|           49|           75|          64|          84|    respiratory rate|          saturation|     test|     test|    O|   test|
|No lotions , crea...|            3|           34|           9|          42|             lotions|           incisions|treatment|  problem|TrNAP|   test|
|Because of expect...|           11|           58|          54|          68|expected long ter...|         a picc line|treatment|treatment|    O|  train|
|She states this l...|           16|           82|          31|          92|    li

In [0]:
data.groupby('dataset').count().show()

+-------+-----+
|dataset|count|
+-------+-----+
|  train|  350|
|   test|  650|
+-------+-----+



In [0]:
data.groupby('rel').count().show()

+-----+-----+
|  rel|count|
+-----+-----+
| TrIP|   14|
| TrAP|  164|
| TeCP|   26|
|    O|  414|
|TrNAP|   14|
| TrCP|   28|
|  PIP|  153|
| TrWP|   11|
| TeRP|  176|
+-----+-----+



In [0]:
from sparknlp_jsl.training import REDatasetHelper

# map entity columns to dataset columns
column_map = {
    "begin1": "firstCharEnt1",
    "end1": "lastCharEnt1",
    "begin2": "firstCharEnt2",
    "end2": "lastCharEnt2",
    "chunk1": "chunk1",
    "chunk2": "chunk2",
    "label1": "label1",
    "label2": "label2"
}

# apply preprocess function to dataframe
data = REDatasetHelper(data).create_annotation_column(
    column_map,
    ner_column_name="train_ner_chunks" # optional, default train_ner_chunks
)

train_data = data.where("dataset='train'")
test_data = data.where("dataset='test'")

In [0]:
data.show(10)

+------------+--------------------+--------------------+-------------+-------+---------+------------+---------+--------------------+--------------------+-----+-------------+
|lastCharEnt1|    train_ner_chunks|              chunk2|firstCharEnt2|dataset|   label2|lastCharEnt2|   label1|              chunk1|            sentence|  rel|firstCharEnt1|
+------------+--------------------+--------------------+-------------+-------+---------+------------+---------+--------------------+--------------------+-----+-------------+
|          64|[{chunk, 49, 64, ...|          saturation|           75|   test|     test|          84|     test|    respiratory rate|VITAL SIGNS - Tem...|    O|           49|
|           9|[{chunk, 3, 9, lo...|           incisions|           34|   test|  problem|          42|treatment|             lotions|No lotions , crea...|TrNAP|            3|
|          54|[{chunk, 11, 54, ...|         a picc line|           58|  train|treatment|          68|treatment|expected long ter..

## Create graph 

We will use `TFGraphBuilder` annotator which can be used to create graphs automatically in the model training pipeline. 

`TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow is available. The graph is stored in the defined folder and loaded by the approach.

You can also create a custom graph by using `tf_graph` module in Spark NLP for Healthcare.

In [0]:
'''
# custom graph

from sparknlp_jsl.training import tf_graph

tf_graph.print_model_params("relation_extraction")

tf_graph.build("relation_extraction", 
             build_params={"input_dim": 10000, 
                          "output_dim": 10, 
                          'batch_norm':1, 
                          "hidden_layers": [300, 200], 
                          "hidden_act": "relu", 
                          'hidden_act_l2':1}, 
                          'model_location'=".", 
                          'model_filename'="re_in6000D_out10.pb")
'''

'''
# ready to use graph
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/RE_in1200D_out20.pb
'''

'\n# ready to use graph\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/RE_in1200D_out20.pb\n'

In [0]:
from sparknlp_jsl.annotator import TFGraphBuilder

In [0]:
graph_folder= "./tf_graphs"

re_graph_builder = TFGraphBuilder()\
    .setModelName("relation_extraction")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"]) \
    .setLabelColumn("rel")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("re_graph.pb")\
    .setHiddenLayers([300, 200])\
    .setHiddenAct("relu")\
    .setHiddenActL2(True)\
    .setHiddenWeightsL2(False)\
    .setBatchNorm(False)

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("sentences")

tokenizer = Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")\

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")
    
dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

# set training params and upload model graph (see ../Healthcare/8.Generic_Classifier.ipynb)
reApproach = RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(80)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setModelFile(f"{graph_folder}/re_graph.pb")\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setOutputLogsPath('/content')

finisher = Finisher()\
    .setInputCols(["relations"])\
    .setOutputCols(["relations_out"])\
    .setCleanAnnotations(False)\
    .setValueSplitSymbol(",")\
    .setAnnotationSplitSymbol(",")\
    .setOutputAsArray(False)

train_pipeline = Pipeline(stages=[
    documenter, 
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    dependency_parser, 
    re_graph_builder,
    reApproach, 
    finisher
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]


In [0]:
%%time 
rel_model = train_pipeline.fit(train_data)


TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


relation_extraction graph exported to ./tf_graphs/re_graph.pb
CPU times: user 2.22 s, sys: 184 ms, total: 2.41 s
Wall time: 10.8 s


In [0]:
rel_model.stages

[DocumentAssembler_6d325790e5ae,
 REGEX_TOKENIZER_da2b7578a090,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 TFGraphBuilderModel_fefa885f9799,
 RelationExtractionModel_b9332c63032a,
 Finisher_1513a52831f0]

In [0]:
rel_model.stages[-2]

RelationExtractionModel_b9332c63032a

In [0]:
# save model
rel_model.stages[-2].write().overwrite().save('/dbfs/custom_RE_model')

### Evaluating The Performance of the Model

We will create a pipeline and use our trained RE model by calling with `.load` method. Then we will get predictions by transforming our test set on this pipeline.

In [0]:
customReModel = RelationExtractionModel.load("/dbfs/custom_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations_pred")\
    .setMaxSyntacticDistance(0)

test_pipeline = Pipeline(stages=[
    documenter, 
    tokenizer, 
    words_embedder, 
    words_embedder, 
    pos_tagger, 
    dependency_parser, 
    customReModel])

test_results = test_pipeline.fit(test_data).transform(test_data)

In [0]:
test_results.show(5)

+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|lastCharEnt1|    train_ner_chunks|              chunk2|firstCharEnt2|dataset| label2|lastCharEnt2|   label1|              chunk1|            sentence|  rel|firstCharEnt1|           sentences|              tokens|          embeddings|            pos_tags|        dependencies|      relations_pred|
+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          64|[{chunk, 49, 64, ...|          saturation|           75|   test|   test|          84|     te

When you check the `relations_pred` column, you can see that some of the relations between the chunk pairs were not detected this time. 

Now we will get metrics using the ground truth (`rel`) and prediction (`relations_pred`) columns in the result dataframe.

In [0]:
pd_test_results = test_results.select('rel', 'relations_pred.result').toPandas()
pd_test_results.head()

,rel,result
0,O,[O]
1,TrNAP,[O]
2,PIP,[PIP]
3,TeRP,[TeRP]
4,TrAP,[TrAP]


We will explode the result column and fill null values as O label.

In [0]:
pd_test_results = pd_test_results.explode("result").fillna("O")
pd_test_results.result.value_counts()

O        321
TrAP     120
PIP      112
TeRP      93
TeCP       3
TrNAP      1
Name: result, dtype: int64

In [0]:
pd_test_results.rel.value_counts()

O        274
TeRP     116
TrAP     108
PIP       88
TrCP      20
TeCP      15
TrIP      11
TrNAP      9
TrWP       9
Name: rel, dtype: int64

In [0]:
from sklearn.metrics import classification_report
print(classification_report(pd_test_results["rel"], pd_test_results["result"]))

              precision    recall  f1-score   support

           O       0.71      0.83      0.77       274
         PIP       0.57      0.73      0.64        88
        TeCP       0.00      0.00      0.00        15
        TeRP       0.81      0.65      0.72       116
        TrAP       0.59      0.66      0.62       108
        TrCP       0.00      0.00      0.00        20
        TrIP       0.00      0.00      0.00        11
       TrNAP       0.00      0.00      0.00         9
        TrWP       0.00      0.00      0.00         9

    accuracy                           0.67       650
   macro avg       0.30      0.32      0.31       650
weighted avg       0.62      0.67      0.64       650



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Training by Setting the Relation Direction

We have `setRelationDirectionCol` parameter that is used during training with a new separate column that specifies the relationship directions. The column should contain one of the following values:

 - `rightwards`: The first entity in the text is also the first argument of the relation (as well as the second entity in the text is the second argument). In other words, the relation arguments are ordered *left to right* in the text.
 - `leftwards`: The first entity in the text is the second argument of the relation (and the second entity in the text is the first argument).
 - `both`: Order doesn't matter (relation is symmetric).

Let's modify our example training dataset accordingly

In [0]:
data.columns

['lastCharEnt1',
 'train_ner_chunks',
 'chunk2',
 'firstCharEnt2',
 'dataset',
 'label2',
 'lastCharEnt2',
 'label1',
 'chunk1',
 'sentence',
 'rel',
 'firstCharEnt1']

In [0]:
@F.udf(T.StringType())
def encodeRelationDirection(rel, begin1, begin2):
    if rel != "O":
        if begin1 > begin2:
            return "leftwards"
        else:
            return "rightwards"
    else:
        return "both"



data = data.withColumn("rel_dir", encodeRelationDirection("rel", "firstCharEnt1", "lastCharEnt2"))

train_data = data.where("dataset='train'")
test_data = data.where("dataset='test'")

In [0]:
train_data.select("chunk1","label1","label2","chunk2","rel","rel_dir").show(10)

+--------------------+---------+---------+--------------------+----+----------+
|              chunk1|   label1|   label2|              chunk2| rel|   rel_dir|
+--------------------+---------+---------+--------------------+----+----------+
|expected long ter...|treatment|treatment|         a picc line|   O|      both|
|    light-headedness|  problem|  problem|         diaphoresis| PIP|rightwards|
| po pain medications|treatment|  problem|            his pain|TrAP| leftwards|
|bilateral pleural...|  problem|  problem|increased work of...| PIP|rightwards|
|    her urine output|     test|  problem|           decreased|TeRP|rightwards|
|his psychiatric i...|  problem|  problem|his neurologic in...| PIP|rightwards|
|   white blood cells|     test|     test|     red blood cells|   O|      both|
|            chloride|     test|     test|                 bun|   O|      both|
|     further work-up|     test|  problem|his neurologic co...|TeCP|rightwards|
|         four liters|treatment|     tes

In [0]:
train_data\
    .selectExpr("concat(label1, \", \", label2) AS args", "rel", "rel_dir")\
    .groupBy("rel", "args", "rel_dir").count().where("count > 10").orderBy("rel").show(1000, truncate=False)

+----+--------------------+----------+-----+
|rel |args                |rel_dir   |count|
+----+--------------------+----------+-----+
|O   |test, test          |both      |59   |
|O   |treatment, treatment|both      |26   |
|O   |problem, problem    |both      |38   |
|PIP |problem, problem    |rightwards|47   |
|PIP |problem, problem    |leftwards |18   |
|TeRP|test, problem       |rightwards|57   |
|TrAP|treatment, problem  |rightwards|29   |
|TrAP|treatment, problem  |leftwards |27   |
+----+--------------------+----------+-----+



Generating a new `RelationExtractionApproach()` with `.setRelationDirectionCol("rel_dir")` and training a new model.

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("sentences")

tokenizer = Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")\

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")
    
dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")
    
reApproach = RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(80)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setMaxSyntacticDistance(10)\
    .setModelFile(f"{graph_folder}/re_graph.pb")\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setOutputLogsPath('/rel_log')\
    .setRelationDirectionCol("rel_dir")

train_pipeline = Pipeline(stages=[
    documenter, 
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    dependency_parser, 
    re_graph_builder,
    reApproach, 
    finisher
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]


In [0]:
%%time 
rel_model = train_pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}
relation_extraction graph exported to ./tf_graphs/re_graph.pb
CPU times: user 733 ms, sys: 5.98 ms, total: 739 ms
Wall time: 8.55 s


In [0]:
rel_model.stages

[DocumentAssembler_f6312914daa9,
 REGEX_TOKENIZER_813a08bfe11e,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 TFGraphBuilderModel_716cc41f199e,
 RelationExtractionModel_d23b857a0102,
 Finisher_1513a52831f0]

In [0]:
# save model
rel_model.stages[-2].write().overwrite().save('dbfs:/databricks/driver/models/rel_dir_RE_model')

### Evaluating The Performance of the Model Trained with Directions

In [0]:
customReModelDir = RelationExtractionModel.load("dbfs:/databricks/driver/models/rel_dir_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations_pred")\
    .setMaxSyntacticDistance(0)

test_pipeline_dir = Pipeline(stages=[
    documenter, 
    tokenizer, 
    words_embedder, 
    words_embedder, 
    pos_tagger, 
    dependency_parser, 
    customReModelDir])

test_results_dir = test_pipeline_dir.fit(test_data).transform(test_data)

In [0]:
customReModel = RelationExtractionModel.load("dbfs:/databricks/driver/models/rel_dir_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations_pred")\
    .setMaxSyntacticDistance(0)

test_pipeline = Pipeline(stages=[
    documenter, 
    tokenizer, 
    words_embedder, 
    words_embedder, 
    pos_tagger, 
    dependency_parser, 
    customReModel])

test_results = test_pipeline.fit(test_data).transform(test_data)

In [0]:
test_results.show(5)

+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|lastCharEnt1|    train_ner_chunks|              chunk2|firstCharEnt2|dataset| label2|lastCharEnt2|   label1|              chunk1|            sentence|  rel|firstCharEnt1|   rel_dir|           sentences|              tokens|          embeddings|            pos_tags|        dependencies|      relations_pred|
+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          64|[{chunk, 49, 64, ...|          saturation|           75|   

When you check the `relations_pred` column, you can see that some of the relations between the chunk pairs were not detected this time. 
Now we will get metrics using the ground truth (`rel`) and prediction (`relations_pred`) columns in the result dataframe.

In [0]:
pd_test_results_dir = test_results_dir.select('rel', 'relations_pred.result').toPandas()
pd_test_results_dir.head()

,rel,result
0,O,[O]
1,TrNAP,[O]
2,PIP,[]
3,TeRP,[TeRP]
4,TrAP,[TrAP]


We will explode the `result` column and fill null values as `O` label and get metrics.

In [0]:
from sklearn.metrics import classification_report

pd_test_results_dir = pd_test_results_dir.explode("result").fillna("O")

print(classification_report(pd_test_results_dir["rel"], pd_test_results_dir["result"]))

              precision    recall  f1-score   support

           O       0.63      0.88      0.74       274
         PIP       0.57      0.48      0.52        88
        TeCP       0.00      0.00      0.00        15
        TeRP       0.78      0.63      0.70       116
        TrAP       0.78      0.68      0.72       108
        TrCP       0.00      0.00      0.00        20
        TrIP       1.00      0.09      0.17        11
       TrNAP       0.00      0.00      0.00         9
        TrWP       0.00      0.00      0.00         9

    accuracy                           0.66       650
   macro avg       0.42      0.31      0.32       650
weighted avg       0.63      0.66      0.63       650



/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Load trained model from disk

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

clinical_ner_tagger = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags") 

loaded_re_Model = RelationExtractionModel.load("dbfs:/databricks/driver/models/custom_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"]) \
    .setOutputCol("relations")\
    .setRelationPairs(["problem-test", "problem-treatment"])\
    .setPredictionThreshold(0.8)\
    .setMaxSyntacticDistance(3)

trained_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    clinical_ner_tagger,
    ner_chunker,
    dependency_parser,
    loaded_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

loaded_re_model = trained_pipeline.fit(empty_data)

ner_clinical download started this may take some time.
[OK!]


In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge.
"""

re_model_light = LightPipeline(loaded_re_model)

annotations = re_model_light.fullAnnotate(text)

rel_df = get_relations_df(annotations)

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,302,318,a body mass index,PROBLEM,322,324,BMI,TEST,O,0.9976318
1,0,322,324,BMI,TEST,392,401,polydipsia,PROBLEM,O,0.9701825
2,0,322,324,BMI,TEST,405,417,poor appetite,PROBLEM,O,0.9453787
3,0,322,324,BMI,TEST,425,432,vomiting,PROBLEM,O,0.97490305
4,1,512,522,amoxicillin,TREATMENT,528,556,a respiratory tract infection,PROBLEM,TrAP,0.9384067
5,2,571,579,metformin,TREATMENT,617,620,T2DM,PROBLEM,O,0.91686404
6,2,583,591,glipizide,TREATMENT,617,620,T2DM,PROBLEM,O,0.9947973
7,2,617,620,T2DM,PROBLEM,626,637,atorvastatin,TREATMENT,O,0.9975405
8,2,617,620,T2DM,PROBLEM,643,653,gemfibrozil,TREATMENT,O,0.9862364
9,2,626,637,atorvastatin,TREATMENT,659,661,HTG,PROBLEM,O,0.9779181


# End-to-end trained Models using BioBERT
#### Latest addition to Spark NLP for Heathcare - (Requires Spark NLP 2.7.3+ and Spark NLP JSL 2.7.3+)

These models are trained as end-to-end bert models using BioBERT and ported in to the Spark NLP ecosystem. 

They offer SOTA performance on most benchmark tasks and outperform our existing Relation Extraction Models.

## ADE ReDL

In [0]:
ner_tagger = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")  

ade_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])

ade_re_model = RelationExtractionDLModel()\
    .pretrained('redl_ade_biobert', 'en', "clinical/models") \
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

ade_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    ade_re_ner_chunk_filter,
    ade_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_model = ade_pipeline.fit(empty_data)

ner_ade_clinical download started this may take some time.
[OK!]
redl_ade_biobert download started this may take some time.
[OK!]


In [0]:
text ="""A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

ade_lmodel = LightPipeline(ade_model)
ade_results = ade_lmodel.fullAnnotate(text)

rel_df = get_relations_df (ade_results)

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,25,32,naproxen,DRUG,137,148,tense bullae,ADE,1,0.9989047
1,0,25,32,naproxen,DRUG,154,210,cutaneous fragility on the face and the back of the hands,ADE,1,0.9989704
2,0,87,95,oxaprozin,DRUG,137,148,tense bullae,ADE,1,0.99895453
3,0,87,95,oxaprozin,DRUG,154,210,cutaneous fragility on the face and the back of the hands,ADE,1,0.99900633


## Clinical ReDL

In [0]:
clinical_ner_tagger = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)\
    .setRelationPairs(["problem-test", "problem-treatment"])# we can set the possible relation pairs (if not set, all the relations will be calculated)
    
clinical_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_clinical_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.9)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    clinical_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_ner_chunk_filter,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_clinical download started this may take some time.
[OK!]
redl_clinical_biobert download started this may take some time.
[OK!]


In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""
lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
1,1,512,522,amoxicillin,TREATMENT,528,556,a respiratory tract infection,PROBLEM,TrAP,0.99863595
2,2,571,579,metformin,TREATMENT,617,620,T2DM,PROBLEM,TrAP,0.99126583
3,2,583,591,glipizide,TREATMENT,617,620,T2DM,PROBLEM,TrAP,0.99036837
4,2,599,611,dapagliflozin,TREATMENT,617,620,T2DM,PROBLEM,TrAP,0.9954288
5,2,599,611,dapagliflozin,TREATMENT,659,661,HTG,PROBLEM,TrAP,0.95774436
6,2,617,620,T2DM,PROBLEM,626,637,atorvastatin,TREATMENT,TrAP,0.9347152
7,2,617,620,T2DM,PROBLEM,643,653,gemfibrozil,TREATMENT,TrAP,0.9791951
8,2,643,653,gemfibrozil,TREATMENT,659,661,HTG,PROBLEM,TrAP,0.97677
9,4,739,758,Physical examination,TEST,796,810,dry oral mucosa,PROBLEM,TeRP,0.9983334
10,5,830,854,her abdominal examination,TEST,875,884,tenderness,PROBLEM,TeRP,0.9946827


## Clinical Temporal Events ReDL

In [0]:
events_ner_tagger = MedicalNerModel.pretrained("ner_events_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")     

events_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")
    
events_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_temporal_events_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.45)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_ner_tagger,
    ner_chunker,
    dependency_parser,
    events_re_ner_chunk_filter,
    events_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_events_clinical download started this may take some time.
[OK!]
redl_temporal_events_biobert download started this may take some time.
[OK!]


In [0]:
text ="""The patient is a 68-year-old Caucasian male with past medical history of diabetes mellitus. 
He was doing fairly well until last week while mowing the lawn, he injured his right foot. """

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df[(rel_df.relation!="O")&(rel_df.entity1!=rel_df.entity2)]


,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,1,100,116,doing fairly well,OCCURRENCE,124,132,last week,DURATION,BEFORE,0.596887
1,1,124,132,last week,DURATION,140,154,mowing the lawn,OCCURRENCE,OVERLAP,0.665236
2,1,140,154,mowing the lawn,OCCURRENCE,160,181,injured his right foot,PROBLEM,OVERLAP,0.536443


In [0]:
text ="""She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed.  
She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 
12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge. 
"""

annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df[(rel_df.relation!="O")]


,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,7,14,admitted,OCCURRENCE,19,43,The John Hopkins Hospital,CLINICAL_DEPT,AFTER,0.974332
1,0,7,14,admitted,OCCURRENCE,45,54,2 days ago,DATE,BEFORE,0.614918
2,0,7,14,admitted,OCCURRENCE,74,102,gestational diabetes mellitus,PROBLEM,BEFORE,0.866523
3,0,19,43,The John Hopkins Hospital,CLINICAL_DEPT,45,54,2 days ago,DATE,OVERLAP,0.716698
4,0,19,43,The John Hopkins Hospital,CLINICAL_DEPT,74,102,gestational diabetes mellitus,PROBLEM,BEFORE,0.591632
5,0,45,54,2 days ago,DATE,74,102,gestational diabetes mellitus,PROBLEM,BEFORE,0.556411
6,1,133,157,the endocrinology service,CLINICAL_DEPT,171,180,discharged,OCCURRENCE,BEFORE,0.661403
7,1,133,157,the endocrinology service,CLINICAL_DEPT,185,194,03/02/2018,DATE,OVERLAP,0.698659
8,1,133,157,the endocrinology service,CLINICAL_DEPT,211,226,insulin glargine,TREATMENT,OVERLAP,0.787637
9,1,133,157,the endocrinology service,CLINICAL_DEPT,242,255,insulin lispro,TREATMENT,OVERLAP,0.748179


## Human Phenotype - Gene ReDL

In [0]:
pgr_ner_tagger = MedicalNerModel.pretrained("ner_human_phenotype_gene_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

pgr_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)
    
pgr_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_human_phenotype_gene_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.6)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    pgr_ner_tagger,
    ner_chunker,
    dependency_parser,
    pgr_re_ner_chunk_filter,
    pgr_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_human_phenotype_gene_clinical download started this may take some time.
[OK!]
redl_human_phenotype_gene_biobert download started this may take some time.
[OK!]


In [0]:
text = "She has a hearing loss and short stature, mutations in the SH3PXD2B gene coding for the Tks4 protein are responsible for the autosomal recessive."

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!=0)]

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,10,21,hearing loss,HP,27,39,short stature,HP,0,0.7657599
1,0,10,21,hearing loss,HP,59,66,SH3PXD2B,GENE,1,0.9079312
2,0,10,21,hearing loss,HP,125,143,autosomal recessive,HP,1,0.859155
3,0,27,39,short stature,HP,59,66,SH3PXD2B,GENE,1,0.88993526


## Drug-Drug Interaction ReDL

In [0]:
ddi_ner_tagger = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ddi_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)
    
ddi_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_drug_drug_interaction_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.9)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ddi_ner_tagger,
    ner_chunker,
    dependency_parser,
    ddi_re_ner_chunk_filter,
    ddi_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_posology download started this may take some time.
[OK!]
redl_drug_drug_interaction_biobert download started this may take some time.
[OK!]


In [0]:
text='When carbamazepine is withdrawn from the combination therapy, aripiprazole dose should then be reduced. \
If additional adrenergic drugs are to be administered by any route, \
they should be used with caution because the pharmacologically predictable sympathetic effects of Metformin may be potentiated'

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,5,17,carbamazepine,DRUG,62,73,aripiprazole,DRUG,DDI-false,0.9168541


## Drug-Protein Interaction ReDL

In [0]:
drugprot_ner_tagger = MedicalNerModel.pretrained("ner_drugprot_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

# Set a filter on pairs of named entities which will be treated as relation candidates
drugprot_re_ner_chunk_filter = RENerChunksFilter()\
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)\
   .setRelationPairs(["checmical-gene", "chemical-gene_and_chemical", "gene_and_chemical-gene"])
    
drugprot_re_Model = RelationExtractionDLModel()\
    .pretrained('redl_drugprot_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.9)\
    .setInputCols(["re_ner_chunks", "sentences"])\
    .setOutputCol("relations")

pipeline = Pipeline(stages=[documenter, sentencer, tokenizer, words_embedder, drugprot_ner_tagger, ner_converter, pos_tagger, dependency_parser, drugprot_re_ner_chunk_filter, drugprot_re_Model])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_drugprot_clinical download started this may take some time.
[OK!]
redl_drugprot_biobert download started this may take some time.
[OK!]


In [0]:
text='''Lipid specific activation of the murine P4-ATPase Atp8a1 (ATPase II). The asymmetric transbilayer distribution of phosphatidylserine (PS) in the mammalian plasma membrane and secretory vesicles is maintained, in part, by an ATP-dependent transporter. This aminophospholipid "flippase" selectively transports PS to the cytosolic leaflet of the bilayer and is sensitive to vanadate, Ca(2+), and modification by sulfhydryl reagents. Although the flippase has not been positively identified, a subfamily of P-type ATPases has been proposed to function as transporters of amphipaths, including PS and other phospholipids. A candidate PS flippase ATP8A1 (ATPase II), originally isolated from bovine secretory vesicles, is a member of this subfamily based on sequence homology to the founding member of the subfamily, the yeast protein Drs2, which has been linked to ribosomal assembly, the formation of Golgi-coated vesicles, and the maintenance of PS asymmetry. To determine if ATP8A1 has biochemical characteristics consistent with a PS flippase, a murine homologue of this enzyme was expressed in insect cells and purified. The purified Atp8a1 is inactive in detergent micelles or in micelles containing phosphatidylcholine, phosphatidic acid, or phosphatidylinositol, is minimally activated by phosphatidylglycerol or phosphatidylethanolamine (PE), and is maximally activated by PS. The selectivity for PS is dependent upon multiple elements of the lipid structure. Similar to the plasma membrane PS transporter, Atp8a1 is activated only by the naturally occurring sn-1,2-glycerol isomer of PS and not the sn-2,3-glycerol stereoisomer. Both flippase and Atp8a1 activities are insensitive to the stereochemistry of the serine headgroup. Most modifications of the PS headgroup structure decrease recognition by the plasma membrane PS flippase. Activation of Atp8a1 is also reduced by these modifications; phosphatidylserine-O-methyl ester, lysophosphatidylserine, glycerophosphoserine, and phosphoserine, which are not transported by the plasma membrane flippase, do not activate Atp8a1. Weakly translocated lipids (PE, phosphatidylhydroxypropionate, and phosphatidylhomoserine) are also weak Atp8a1 activators. However, N-methyl-phosphatidylserine, which is transported by the plasma membrane flippase at a rate equivalent to PS, is incapable of activating Atp8a1 activity. These results indicate that the ATPase activity of the secretory granule Atp8a1 is activated by phospholipids binding to a specific site whose properties (PS selectivity, dependence upon glycerol but not serine, stereochemistry, and vanadate sensitivity) are similar to, but distinct from, the properties of the substrate binding site of the plasma membrane flippase.'''

lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [0]:
results[0].keys()

dict_keys(['sentences', 'document', 'ner_chunks', 'ner_tags', 'relations', 'tokens', 'embeddings', 'pos_tags', 're_ner_chunks', 'dependencies'])

In [0]:
results[0]["ner_chunks"]

[Annotation(chunk, 33, 48, murine P4-ATPase, {'chunk': '0', 'confidence': '0.75505', 'ner_source': 'ner_chunks', 'entity': 'GENE', 'sentence': '0'}, []),
 Annotation(chunk, 50, 55, Atp8a1, {'chunk': '1', 'confidence': '0.9911', 'ner_source': 'ner_chunks', 'entity': 'GENE', 'sentence': '0'}, []),
 Annotation(chunk, 58, 66, ATPase II, {'chunk': '2', 'confidence': '0.8749', 'ner_source': 'ner_chunks', 'entity': 'GENE', 'sentence': '0'}, []),
 Annotation(chunk, 114, 131, phosphatidylserine, {'chunk': '3', 'confidence': '0.9794', 'ner_source': 'ner_chunks', 'entity': 'CHEMICAL', 'sentence': '1'}, []),
 Annotation(chunk, 134, 135, PS, {'chunk': '4', 'confidence': '0.7845', 'ner_source': 'ner_chunks', 'entity': 'CHEMICAL', 'sentence': '1'}, []),
 Annotation(chunk, 224, 248, ATP-dependent transporter, {'chunk': '5', 'confidence': '0.77744997', 'ner_source': 'ner_chunks', 'entity': 'GENE_AND_CHEMICAL', 'sentence': '1'}, []),
 Annotation(chunk, 256, 282, aminophospholipid "flippase, {'chunk': '6

In [0]:
results[0]["relations"]

[Annotation(category, 114, 248, SUBSTRATE, {'chunk2': 'ATP-dependent transporter', 'confidence': '0.9931178', 'entity2_end': '248', 'syntactic_distance': '3', 'chunk1': 'phosphatidylserine', 'entity1': 'CHEMICAL', 'entity2_begin': '224', 'chunk2_confidence': '0.77744997', 'entity1_begin': '114', 'sentence': '1', 'entity1_end': '131', 'entity2': 'GENE_AND_CHEMICAL', 'chunk1_confidence': '0.9794', 'context': 'The asymmetric transbilayer distribution of phosphatidylserine (PS) in the mammalian plasma membrane and secretory vesicles is maintained, in part, by an ATP-dependent transporter.'}, []),
 Annotation(category, 134, 248, SUBSTRATE, {'chunk2': 'ATP-dependent transporter', 'confidence': '0.9978284', 'entity2_end': '248', 'syntactic_distance': '4', 'chunk1': 'PS', 'entity1': 'CHEMICAL', 'entity2_begin': '224', 'chunk2_confidence': '0.77744997', 'entity1_begin': '134', 'sentence': '1', 'entity1_end': '135', 'entity2': 'GENE_AND_CHEMICAL', 'chunk1_confidence': '0.7845', 'context': 'The a

In [0]:
rel_df = get_relations_df (results)

rel_df

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,1,114,131,phosphatidylserine,CHEMICAL,224,248,ATP-dependent transporter,GENE_AND_CHEMICAL,SUBSTRATE,0.9931178
1,1,134,135,PS,CHEMICAL,224,248,ATP-dependent transporter,GENE_AND_CHEMICAL,SUBSTRATE,0.9978284
2,2,256,282,"aminophospholipid ""flippase",GENE_AND_CHEMICAL,308,309,PS,CHEMICAL,SUBSTRATE,0.9968598


## Chemical–Protein Interactions (ChemProt) ReDL

In [0]:
chemprot_ner_tagger = MedicalNerModel.pretrained("ner_chemprot_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

chemprot_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)
    
chemprot_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_chemprot_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.9)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    chemprot_ner_tagger,
    ner_chunker,
    dependency_parser,
    chemprot_re_ner_chunk_filter,
    chemprot_re_Model
])


empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_chemprot_clinical download started this may take some time.
[OK!]
redl_chemprot_biobert download started this may take some time.
[OK!]


In [0]:
text='''
In this study, we examined the effects of mitiglinide on various cloned K(ATP) channels (Kir6.2/SUR1, Kir6.2/SUR2A, and Kir6.2/SUR2B) reconstituted in COS-1 cells, and compared them to another meglitinide-related compound, nateglinide. Patch-clamp analysis using inside-out recording configuration showed that mitiglinide inhibits the Kir6.2/SUR1 channel currents in a dose-dependent manner (IC50 value, 100 nM) but does not significantly inhibit either Kir6.2/SUR2A or Kir6.2/SUR2B channel currents even at high doses (more than 10 microM). Nateglinide inhibits Kir6.2/SUR1 and Kir6.2/SUR2B channels at 100 nM, and inhibits Kir6.2/SUR2A channels at high concentrations (1 microM). Binding experiments on mitiglinide, nateglinide, and repaglinide to SUR1 expressed in COS-1 cells revealed that they inhibit the binding of [3H]glibenclamide to SUR1 (IC50 values: mitiglinide, 280 nM; nateglinide, 8 microM; repaglinide, 1.6 microM), suggesting that they all share a glibenclamide binding site. The insulin responses to glucose, mitiglinide, tolbutamide, and glibenclamide in MIN6 cells after chronic mitiglinide, nateglinide, or repaglinide treatment were comparable to those after chronic tolbutamide and glibenclamide treatment. These results indicate that, similar to the sulfonylureas, mitiglinide is highly specific to the Kir6.2/SUR1 complex, i.e., the pancreatic beta-cell K(ATP) channel, and suggest that mitiglinide may be a clinically useful anti-diabetic drug.
'''
lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df[rel_df.entity1!=rel_df.entity2]

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,43,53,mitiglinide,CHEMICAL,80,87,channels,GENE-N,CPR:2,0.9993729
1,0,80,87,channels,GENE-N,224,234,nateglinide,CHEMICAL,CPR:2,0.99562293
4,3,706,716,mitiglinide,CHEMICAL,751,754,SUR1,GENE-Y,CPR:2,0.99915874
6,3,719,729,nateglinide,CHEMICAL,751,754,SUR1,GENE-Y,CPR:2,0.9987863
7,3,736,746,repaglinide,CHEMICAL,751,754,SUR1,GENE-Y,CPR:2,0.99850047
8,3,823,839,[3H]glibenclamide,CHEMICAL,844,847,SUR1,GENE-Y,CPR:2,0.9979025
10,6,998,1004,insulin,GENE-N,1019,1025,glucose,CHEMICAL,CPR:2,0.97875994
11,6,998,1004,insulin,GENE-N,1028,1038,mitiglinide,CHEMICAL,CPR:2,0.9898919
12,6,998,1004,insulin,GENE-N,1041,1051,tolbutamide,CHEMICAL,CPR:2,0.98977774
13,6,998,1004,insulin,GENE-N,1058,1070,glibenclamide,CHEMICAL,CPR:2,0.99023926


# Relation Extraction Across Sentences

We can extract relations across the sentences in the document by dropping the `SentenceDetector` in the pipeline. Since ReDL models are trained with BERT embeddings, they can perform better than their RE versions. **BUT** it all depends on the data that we are working on, so we can try different models and use the best one in our pipeline.

In [0]:
import pandas as pd

def get_only_relations_df(results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity1'], 
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['entity2'],
          rel.result, 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['entity1_begin','entity1_end','chunk1', 'entity1', 'entity2_begin','entity2_end','chunk2', 'entity2', 'relation', 'confidence'])

  return rel_df

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document", "tokens"])\
    .setOutputCol("embeddings")

ner_tagger = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("document", "tokens", "embeddings")\
    .setOutputCol("ner_tags")

ner_chunker = NerConverterInternal()\
    .setInputCols(["document", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["document", "tokens"])\
    .setOutputCol("pos_tags")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["document", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

ade_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(0)\
    .setRelationPairs(["drug-ade, ade-drug"])

ade_re_model = RelationExtractionDLModel()\
    .pretrained('redl_ade_biobert', 'en', "clinical/models") \
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunks", "document"]) \
    .setOutputCol("relations")

ade_pipeline = Pipeline(stages=[
    documenter,
    tokenizer, 
    words_embedder, 
    ner_tagger,
    ner_chunker,
    pos_tagger, 
    dependency_parser,
    ade_re_ner_chunk_filter,
    ade_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
ade_model = ade_pipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_ade_clinical download started this may take some time.
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
redl_ade_biobert download started this may take some time.
[OK!]


In [0]:
ade_lmodel = LightPipeline(ade_model)

text ="""A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis. 
They both presented with tense bullae and cutaneous fragility on the face and the back of the hands. 
"""

ade_results = ade_lmodel.fullAnnotate(text)
rel_df = get_only_relations_df (ade_results)

rel_df

,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,25,32,naproxen,DRUG,149,160,tense bullae,ADE,1,0.9988153
1,25,32,naproxen,DRUG,166,222,cutaneous fragility on the face and the back of the hands,ADE,1,0.9989286
2,87,95,oxaprozin,DRUG,149,160,tense bullae,ADE,1,0.9987915
3,87,95,oxaprozin,DRUG,166,222,cutaneous fragility on the face and the back of the hands,ADE,1,0.9988927


In [0]:
text = """The patient continued to receive regular insulin 4 times per day over the following 3 years with only occasional hives. 
Two days ago, suddenly severe urticaria, angioedema, and occasional wheezing began."""

ade_results = ade_lmodel.fullAnnotate(text)
rel_df = get_only_relations_df (ade_results)

rel_df

,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,41,47,insulin,DRUG,113,117,hives,ADE,1,0.99795294
1,41,47,insulin,DRUG,144,159,severe urticaria,ADE,1,0.9975689
2,41,47,insulin,DRUG,162,171,angioedema,ADE,1,0.9965976
3,41,47,insulin,DRUG,178,196,occasional wheezing,ADE,1,0.99725205


# Merging Multiple RE Model Results

We can merge multiple RE model results by using `AnnotationMerger()`. <br/>
Now, we will build a pipeline consisting of `posology_re`, `re_ade_clinical` models and `AnnotationMerger()` to merge these RE models' results.

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

pos_ner_tagger = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_pos")

pos_ner_chunker = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ner_pos"])\
    .setOutputCol("pos_ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

pos_reModel = RelationExtractionModel()\
    .pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "pos_ner_chunks", "dependencies"])\
    .setOutputCol("pos_relations")\
    .setMaxSyntacticDistance(4)

ade_ner_tagger = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ade_ner_tags")  

ade_ner_chunker = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ade_ner_tags"])\
    .setOutputCol("ade_ner_chunks")

ade_reModel = RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ade_ner_chunks", "dependencies"])\
    .setOutputCol("ade_relations")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])

annotation_merger = AnnotationMerger()\
    .setInputCols("ade_relations", "pos_relations")\
    .setInputType("category")\
    .setOutputCol("all_relations")

merger_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    pos_ner_tagger,
    pos_ner_chunker,
    dependency_parser,
    pos_reModel,
    ade_ner_tagger,
    ade_ner_chunker,
    ade_reModel,
    annotation_merger
])

empty_df= spark.createDataFrame([[""]]).toDF("text")
merger_model= merger_pipeline.fit(empty_df)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
ner_posology download started this may take some time.
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
ner_ade_clinical download started this may take some time.
[OK!]
re_ade_clinical download started this may take some time.
[OK!]


In [0]:
text = """
The patient was prescribed 1 unit of naproxen for 5 days after meals for chronic low back pain. The patient was also given 1 unit of oxaprozin daily for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.. 
"""
data = spark.createDataFrame([[text]]).toDF("text")

result = merger_model.transform(data)
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|           sentences|              tokens|          embeddings|            pos_tags|             ner_pos|      pos_ner_chunks|        dependencies|       pos_relations|        ade_ner_tags|      ade_ner_chunks|       ade_relations|       all_relations|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|\nThe patient was...|[{document, 0, 26...|[{document, 1, 95...|[{token, 1, 3, Th...|[{word_embeddings...|[{pos,

In [0]:
from pyspark.sql import functions as F

result_df = result.select(F.explode(F.arrays_zip(result.pos_relations.result,
                                                 result.pos_relations.metadata,
                                                 result.ade_relations.result,
                                                 result.ade_relations.metadata,
                                                 result.all_relations.result,
                                                 result.all_relations.metadata)).alias("cols")) \
                  .select(
                          F.expr("cols['0']").alias("pos_relations"),\
                          F.expr("cols['1']['entity1']").alias("pos_relations_entity1"),\
                          F.expr("cols['1']['chunk1']" ).alias("pos_relations_chunk1" ),\
                          F.expr("cols['1']['entity2']").alias("pos_relations_entity2"),\
                          F.expr("cols['1']['chunk2']" ).alias("pos_relations_chunk2" ),\
                          F.expr("cols['2']").alias("ade_relations"),\
                          F.expr("cols['3']['entity1']").alias("ade_relations_entity1"),\
                          F.expr("cols['3']['chunk1']" ).alias("ade_relations_chunk1" ),\
                          F.expr("cols['3']['entity2']").alias("ade_relations_entity2"),\
                          F.expr("cols['3']['chunk2']" ).alias("ade_relations_chunk2" ),\
                          F.expr("cols['4']").alias("all_relations"),\
                          F.expr("cols['5']['entity1']").alias("all_relations_entity1"),\
                          F.expr("cols['5']['chunk1']" ).alias("all_relations_chunk1" ),\
                          F.expr("cols['5']['entity2']").alias("all_relations_entity2"),\
                          F.expr("cols['5']['chunk2']" ).alias("all_relations_chunk2" )
                          )

result_df.show()

+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+
| pos_relations|pos_relations_entity1|pos_relations_chunk1|pos_relations_entity2|pos_relations_chunk2|ade_relations|ade_relations_entity1|ade_relations_chunk1|ade_relations_entity2|ade_relations_chunk2| all_relations|all_relations_entity1|all_relations_chunk1|all_relations_entity2|all_relations_chunk2|
+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+
|   DOSAGE-DRUG|               DOSAGE|              1 unit|                 DRUG|       

Now, we will build a LightPipeline and visualize the results by using `RelationExtractionVisualizer`

In [0]:
lp_merger = LightPipeline(merger_model)
lp_res = lp_merger.fullAnnotate(text)[0]

In [0]:
from sparknlp_display import RelationExtractionVisualizer


visualizer = RelationExtractionVisualizer()
vis = visualizer.display(lp_res, 'all_relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

The patient was prescribed 1 unit DOSAGE of naproxen DRUG for 5 days DURATION after meals for chronic low back pain. The patient was also given 1 unit DOSAGE of oxaprozin DRUG daily FREQUENCY for rheumatoid arthritis presented with tense bullae ADE and cutaneous fragility on the face and the back of the hands ADE .. <polyline fill="none" points="533.0,155.0 533.8877551020407,153.7960798646822 534.7755102040815,152.59312871337622 535.6632653061225,151.39211553009378 536.5510204081633,150.19400929884657 537.4387755102041,148.99977900364644 538.3265306122448,147.81039362850512 539.2142857142859,146.62682215743445 540.1020408163266,145.45003357444605 540.9897959183675,144.28099686355176 541.8775510204081,143.12068100876337 542.7653061224489,141.97005499409258 543.6530612244896,140.83008780355124 544.5408163265306,139.70174842115105 545.4285714285714,138.5860058309038 546.3163265306124,137.48382901682123 547.2040816326531,136.39618696291512 548.091836734694,135.32404865319725 548.9795918367347,134.2683830716793 549.8673469387754,133.23015920237316 550.7551020408163,132.2103460292905 551.6428571428571,131.20991253644314 552.530612244898,130.22982770784282 553.4183673469388,129.2710605275013 554.3061224489796,128.33457997943034 555.1938775510205,127.42135504764173 556.0816326530612,126.53235471614718 556.969387755102,125.6685479689585 557.8571428571429,124.83090379008749 558.7448979591837,124.02039116354581 559.6326530612245,123.2379790733453 560.5204081632653,122.48463650349771 561.4081632653061,121.76133243801479 562.2959183673469,121.06903586090831 563.1836734693878,120.40871575619002 564.0714285714286,119.78134110787173 564.9591836734694,119.18788089996517 565.8469387755101,118.62930411648207 566.7346938775511,118.10657974143427 567.6224489795918,117.62067675883348 568.5102040816327,117.17256415269148 569.3979591836735,116.76321090702004 570.2857142857143,116.39358600583091 571.1734693877551,116.06465843313586 572.0612244897959,115.77739717294665 572.9489795918367,115.53277120927505 573.8367346938776,115.33174952613282 574.7244897959184,115.17530110753174 575.6122448979592,115.06439493748353 576.5,115.0 576.5,115.0 577.3877551020407,114.9827622844223 578.2755102040816,115.01203580140927 579.1632653061224,115.08685156694916 580.0510204081633,115.20624059703015 580.938775510204,115.36923390764053 581.8265306122448,115.57486251476851 582.7142857142859,115.82215743440234 583.6020408163265,116.11014968253025 584.4897959183673,116.43787027514047 585.377551020408,116.80435022822121 586.265306122449,117.20862055776078 587.1530612244898,117.64971227974738 588.0408163265305,118.12665641016923 588.9285714285714,118.63848396501459 589.8163265306123,119.18422596027166 590.7040816326531,119.7629134119287 591.591836734694,120.37357733597398 592.4795918367347,121.01524874839565 593.3673469387755,121.68695866518203 594.2551020408163,122.3877381023213 595.1428571428571,123.11661807580174 596.0306122448981,123.87262960161156 596.9183673469387,124.654803695739 597.8061224489796,125.46217137417233 598.6938775510204,126.29376365289971 599.5816326530612,127.14861154790944 600.469387755102,128.02574607518974 601.3571428571429,128.92419825072886 602.2448979591837,129.842999090515 603.1326530612246,130.7811796105364 604.0204081632654,131.73777082678137 604.908163265306,132.71180375523804 605.795918367347,133.7023094118947 606.6836734693877,134.7083188127396 607.5714285714287,135.72886297376095 608.4591836734694,136.76297291094698 609.3469387755102,137.80967964028594 610.2346938775511,138.86801417776604 611.1224489795918,139.9370075393756 612.0102040816327,141.01569074110276 612.8979591836735,142.1030947989358 613.7857142857142,143.19825072886297 614.6734693877551,144.30018954687245 615.5612244897959,145.40794226895255 616.4489795918367,146.52053991109145 617.3367346938776,147.63701348927742 618.2244897959183,148.75639401949866 619.1122448979592,149.87771251774345 620.0,151.0" stroke="slategray" stroke-width="1" /> DRUG-FREQUENCY <polyline fill="none" poin

# Clinical Knowledge Graph Creation using Neo4j

We have recently published a [Medium article](https://medium.com/spark-nlp/creating-knowledge-graph-by-spark-nlp-neo4j-9d18706aa08b) explains "How to create a  clinical knowledge graph using Spark NLP and Neo4j." and for detailed information, you can play with [the notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/10.2.Clinical_RE_Knowledge_Graph_with_Neo4j.ipynb). Briefly, in this article, we built a Knowledge Graph (KG) using Spark NLP Relation Extraction (RE) Models and Neo4j. Some of the Spark NLP users ask "How can I use the RE results?". First, you can use the results to feed downstream pipelines. Secondly, you can create a KG to try to get insights. The article explains how to accomplish the second option.

# Relation Extraction Model-NER Model-Relation Pairs Table

In the table below, you can find the Relationship Extraction models and the most appropriate NER models and Relationship Pairs that can be used to get the most efficient results when using these models. <br/>
Also, you can reach this table in [Sparknlp for Healthcare Documentation](https://nlp.johnsnowlabs.com/docs/en/best_practices_pretrained_models#relation-extraction-models-and-relation-pairs-table)

|    | RELATION EXTRACTION MODEL            | RELATION EXTRACTION MODEL LABELS                                        | NER MODEL                         | RELATION PAIRS                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|---:|:-------------------------------------|:------------------------------------------------------------------------|:----------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | re_bodypart_proceduretest            | 0-1                                                                     | ner_jsl                           | ["external_body_part_or_region-test", <br/> "test-external_body_part_or_region",<br/>"internal_organ_or_component-test",<br/>"test-internal_organ_or_component",<br/>"external_body_part_or_region-procedure",<br/>"procedure-external_body_part_or_region",<br/>"procedure-internal_organ_or_component",<br/>"internal_organ_or_component-procedure"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  1 | re_ade_clinical                      | 0-1                                                                     | ner_ade_clinical                  | ["ade-drug",<br/> "drug-ade"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|  2 | redl_chemprot_biobert                | CPR:1, CPR:2, CPR:3, CPR:4, CPR:5, CPR:6, CPR:7, CPR:8, CPR:9, CPR:10   | ner_chemprot_clinical             |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  3 | re_human_phenotype_gene_clinical     | 0-1                                                                     | ner_human_phenotype_gene_clinical |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  4 | re_bodypart_directions               | 0-1                                                                     | ner_jsl                           | ["direction-external_body_part_or_region",<br/>"external_body_part_or_region-direction",<br/>"direction-internal_organ_or_component",<br/>"internal_organ_or_component-direction"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|  5 | re_bodypart_problem                  | 0-1                                                                     | ner_jsl                           | ["internal_organ_or_component-cerebrovascular_disease", "cerebrovascular_disease-internal_organ_or_component",<br/>"internal_organ_or_component-communicable_disease", "communicable_disease-internal_organ_or_component",<br/>"internal_organ_or_component-diabetes", "diabetes-internal_organ_or_component",<br/>"internal_organ_or_component-disease_syndrome_disorder", "disease_syndrome_disorder-internal_organ_or_component",<br/>"internal_organ_or_component-ekg_findings", "ekg_findings-internal_organ_or_component",<br/>"internal_organ_or_component-heart_disease", "heart_disease-internal_organ_or_component",<br/>"internal_organ_or_component-hyperlipidemia", "hyperlipidemia-internal_organ_or_component",<br/>"internal_organ_or_component-hypertension", "hypertension-internal_organ_or_component",<br/>"internal_organ_or_component-imagingfindings", "imagingfindings-internal_organ_or_component",<br/>"internal_organ_or_component-injury_or_poisoning", "injury_or_poisoning-internal_organ_or_component",<br/>"internal_organ_or_component-kidney_disease", "kidney_disease-internal_organ_or_component",<br/>"internal_organ_or_component-oncological", "oncological-internal_organ_or_component",<br/>"internal_organ_or_component-psychological_condition", "psychological_condition-internal_organ_or_component",<br/>"internal_organ_or_component-symptom", "symptom-internal_organ_or_component",<br/>"internal_organ_or_component-vs_finding", "vs_finding-internal_organ_or_component",<br/>"external_body_part_or_region-communicable_disease", "communicable_disease-external_body_part_or_region",<br/>"external_body_part_or_region-diabetes", "diabetes-external_body_part_or_region",<br/>"external_body_part_or_region-disease_syndrome_disorder", "disease_syndrome_disorder-external_body_part_or_region",<br/>"external_body_part_or_region-hypertension", "hypertension-external_body_part_or_region",<br/>"external_body_part_or_region-imagingfindings", "imagingfindings-external_body_part_or_region",<br/>"external_body_part_or_region-injury_or_poisoning", "injury_or_poisoning-external_body_part_or_region",<br/>"external_body_part_or_region-obesity", "obesity-external_body_part_or_region",<br/>"external_body_part_or_region-oncological", "oncological-external_body_part_or_region",<br/>"external_body_part_or_region-overweight", "overweight-external_body_part_or_region",<br/>"external_body_part_or_region-symptom", "symptom-external_body_part_or_region",<br/>"external_body_part_or_region-vs_finding", "vs_finding-external_body_part_or_region"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|  6 | re_drug_drug_interaction_clinical    | DDI-advise, DDI-effect, DDI-mechanism, DDI-int, DDI-false               | ner_posology                      | ["drug-drug"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|  7 | re_clinical                          | TrIP, TrWP, TrCP, TrAP, TrAP, TeRP, TeCP, PIP                           | ner_clinical                      |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  8 | re_temporal_events_clinical          | AFTER, BEFORE, OVERLAP                                                  | ner_events_clinical               |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  9 | re_temporal_events_enriched_clinical | BEFORE, AFTER, SIMULTANEOUS, BEGUN_BY, ENDED_BY, DURING, BEFORE_OVERLAP | ner_events_clinical               |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 10 | re_test_problem_finding              | 0-1                                                                     | ner_jsl                           | ["test-cerebrovascular_disease", "cerebrovascular_disease-test",<br/>"test-communicable_disease", "communicable_disease-test",<br/>"test-diabetes", "diabetes-test",<br/>"test-disease_syndrome_disorder", "disease_syndrome_disorder-test",<br/>"test-heart_disease", "heart_disease-test",<br/>"test-hyperlipidemia", "hyperlipidemia-test",<br/>"test-hypertension", "hypertension-test",<br/>"test-injury_or_poisoning", "injury_or_poisoning-test",<br/>"test-kidney_disease", "kidney_disease-test",<br/>"test-obesity", "obesity-test",<br/>"test-oncological", "oncological-test",<br/>"test-psychological_condition", "psychological_condition-test",<br/>"test-symptom", "symptom-test",<br/>"ekg_findings-disease_syndrome_disorder", "disease_syndrome_disorder-ekg_findings",<br/>"ekg_findings-heart_disease", "heart_disease-ekg_findings",<br/>"ekg_findings-symptom", "symptom-ekg_findings",<br/>"imagingfindings-cerebrovascular_disease", "cerebrovascular_disease-imagingfindings",<br/>"imagingfindings-communicable_disease", "communicable_disease-imagingfindings",<br/>"imagingfindings-disease_syndrome_disorder", "disease_syndrome_disorder-imagingfindings",<br/>"imagingfindings-heart_disease", "heart_disease-imagingfindings",<br/>"imagingfindings-hyperlipidemia", "hyperlipidemia-imagingfindings",<br/>"imagingfindings-hypertension", "hypertension-imagingfindings",<br/>"imagingfindings-injury_or_poisoning", "injury_or_poisoning-imagingfindings",<br/>"imagingfindings-kidney_disease", "kidney_disease-imagingfindings",<br/>"imagingfindings-oncological", "oncological-imagingfindings",<br/>"imagingfindings-psychological_condition", "psychological_condition-imagingfindings",<br/>"imagingfindings-symptom", "symptom-imagingfindings",<br/>"vs_finding-cerebrovascular_disease", "cerebrovascular_disease-vs_finding",<br/>"vs_finding-communicable_disease", "communicable_disease-vs_finding",<br/>"vs_finding-diabetes", "diabetes-vs_finding",<br/>"vs_finding-disease_syndrome_disorder", "disease_syndrome_disorder-vs_finding",<br/>"vs_finding-heart_disease", "heart_disease-vs_finding",<br/>"vs_finding-hyperlipidemia", "hyperlipidemia-vs_finding",<br/>"vs_finding-hypertension", "hypertension-vs_finding",<br/>"vs_finding-injury_or_poisoning", "injury_or_poisoning-vs_finding",<br/>"vs_finding-kidney_disease", "kidney_disease-vs_finding",<br/>"vs_finding-obesity", "obesity-vs_finding",<br/>"vs_finding-oncological", "oncological-vs_finding",<br/>"vs_finding-overweight", "overweight-vs_finding",<br/>"vs_finding-psychological_condition", "psychological_condition-vs_finding",<br/>"vs_finding-symptom", "symptom-vs_finding"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| 11 | re_test_result_date                  | is_finding_of, is_result_of, is_date_of, O                              | ner_jsl                           | ["test-test_result", "test_result-test",<br/>"test-date", "date-test",<br/>"test-imagingfindings", "imagingfindings-test",<br/>"test-ekg_findings", "ekg_findings-test",<br/>"date-test_result", "test_result-date",<br/>"date-imagingfindings", "imagingfindings-date",<br/>"date-ekg_findings", "ekg_findings-date"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 12 | re_date_clinical                     | 0-1                                                                     | ner_jsl                           | ["date-admission_discharge", "admission_discharge-date",<br/>"date-alcohol", "alcohol-date",<br/>"date-allergen", "allergen-date",<br/>"date-bmi", "bmi-date",<br/>"date-birth_entity", "birth_entity-date",<br/>"date-blood_pressure", "blood_pressure-date",<br/>"date-cerebrovascular_disease", "cerebrovascular_disease-date",<br/>"date-clinical_dept", "clinical_dept-date",<br/>"date-communicable_disease", "communicable_disease-date",<br/>"date-death_entity", "death_entity-date",<br/>"date-diabetes", "diabetes-date",<br/>"date-diet", "diet-date",<br/>"date-disease_syndrome_disorder", "disease_syndrome_disorder-date",<br/>"date-drug_brandname", "drug_brandname-date",<br/>"date-drug_ingredient", "drug_ingredient-date",<br/>"date-ekg_findings", "ekg_findings-date",<br/>"date-external_body_part_or_region", "external_body_part_or_region-date",<br/>"date-fetus_newborn", "fetus_newborn-date",<br/>"date-hdl", "hdl-date",<br/>"date-heart_disease", "heart_disease-date",<br/>"date-height", "height-date",<br/>"date-hyperlipidemia", "hyperlipidemia-date",<br/>"date-hypertension", "hypertension-date",<br/>"date-imagingfindings", "imagingfindings-date",<br/>"date-imaging_technique", "imaging_technique-date",<br/>"date-injury_or_poisoning", "injury_or_poisoning-date",<br/>"date-internal_organ_or_component", "internal_organ_or_component-date",<br/>"date-kidney_disease", "kidney_disease-date",<br/>"date-ldl", "ldl-date",<br/>"date-modifier", "modifier-date",<br/>"date-o2_saturation", "o2_saturation-date",<br/>"date-obesity", "obesity-date",<br/>"date-oncological", "oncological-date",<br/>"date-overweight", "overweight-date",<br/>"date-oxygen_therapy", "oxygen_therapy-date",<br/>"date-pregnancy", "pregnancy-date",<br/>"date-procedure", "procedure-date",<br/>"date-psychological_condition", "psychological_condition-date",<br/>"date-pulse", "pulse-date",<br/>"date-respiration", "respiration-date",<br/>"date-smoking", "smoking-date",<br/>"date-substance", "substance-date",<br/>"date-substance_quantity", "substance_quantity-date",<br/>"date-symptom", "symptom-date",<br/>"date-temperature", "temperature-date",<br/>"date-test", "test-date",<br/>"date-test_result", "test_result-date",<br/>"date-total_cholesterol", "total_cholesterol-date",<br/>"date-treatment", "treatment-date",<br/>"date-triglycerides", "triglycerides-date",<br/>"date-vs_finding", "vs_finding-date",<br/>"date-vaccine", "vaccine-date",<br/>"date-vital_signs_header", "vital_signs_header-date",<br/>"date-weight", "weight-date",<br/>"time-admission_discharge", "admission_discharge-time",<br/>"time-alcohol", "alcohol-time",<br/>"time-allergen", "allergen-time",<br/>"time-bmi", "bmi-time",<br/>"time-birth_entity", "birth_entity-time",<br/>"time-blood_pressure", "blood_pressure-time",<br/>"time-cerebrovascular_disease", "cerebrovascular_disease-time",<br/>"time-clinical_dept", "clinical_dept-time",<br/>"time-communicable_disease", "communicable_disease-time",<br/>"time-death_entity", "death_entity-time",<br/>"time-diabetes", "diabetes-time",<br/>"time-diet", "diet-time",<br/>"time-disease_syndrome_disorder", "disease_syndrome_disorder-time",<br/>"time-drug_brandname", "drug_brandname-time",<br/>"time-drug_ingredient", "drug_ingredient-time",<br/>"time-ekg_findings", "ekg_findings-time",<br/>"time-external_body_part_or_region", "external_body_part_or_region-time",<br/>"time-fetus_newborn", "fetus_newborn-time",<br/>"time-hdl", "hdl-time",<br/>"time-heart_disease", "heart_disease-time",<br/>"time-height", "height-time",<br/>"time-hyperlipidemia", "hyperlipidemia-time",<br/>"time-hypertension", "hypertension-time",<br/>"time-imagingfindings", "imagingfindings-time",<br/>"time-imaging_technique", "imaging_technique-time",<br/>"time-injury_or_poisoning", "injury_or_poisoning-time",<br/>"time-internal_organ_or_component", "internal_organ_or_component-time",<br/>"time-kidney_disease", "kidney_disease-time",<br/>"time-ldl", "ldl-time",<br/>"time-modifier", "modifier-time",<br/>"time-o2_saturation", "o2_saturation-time",<br/>"time-obesity", "obesity-time",<br/>"time-oncological", "oncological-time",<br/>"time-overweight", "overweight-time",<br/>"time-oxygen_therapy", "oxygen_therapy-time",<br/>"time-pregnancy", "pregnancy-time",<br/>"time-procedure", "procedure-time",<br/>"time-psychological_condition", "psychological_condition-time",<br/>"time-pulse", "pulse-time",<br/>"time-respiration", "respiration-time",<br/>"time-smoking", "smoking-time",<br/>"time-substance", "substance-time",<br/>"time-substance_quantity", "substance_quantity-time",<br/>"time-symptom", "symptom-time",<br/>"time-temperature", "temperature-time",<br/>"time-test", "test-time",<br/>"time-test_result", "test_result-time",<br/>"time-total_cholesterol", "total_cholesterol-time",<br/>"time-treatment", "treatment-time",<br/>"time-triglycerides", "triglycerides-time",<br/>"time-vs_finding", "vs_finding-time",<br/>"time-vaccine", "vaccine-time",<br/>"time-vital_signs_header", "vital_signs_header-time",<br/>"time-weight", "weight-time",<br/>"relativedate-admission_discharge", "admission_discharge-relativedate",<br/>"relativedate-alcohol", "alcohol-relativedate",<br/>"relativedate-allergen", "allergen-relativedate",<br/>"relativedate-bmi", "bmi-relativedate",<br/>"relativedate-birth_entity", "birth_entity-relativedate",<br/>"relativedate-blood_pressure", "blood_pressure-relativedate",<br/>"relativedate-cerebrovascular_disease", "cerebrovascular_disease-relativedate",<br/>"relativedate-clinical_dept", "clinical_dept-relativedate",<br/>"relativedate-communicable_disease", "communicable_disease-relativedate",<br/>"relativedate-death_entity", "death_entity-relativedate",<br/>"relativedate-diabetes", "diabetes-relativedate",<br/>"relativedate-diet", "diet-relativedate",<br/>"relativedate-disease_syndrome_disorder", "disease_syndrome_disorder-relativedate",<br/>"relativedate-drug_brandname", "drug_brandname-relativedate",<br/>"relativedate-drug_ingredient", "drug_ingredient-relativedate",<br/>"relativedate-ekg_findings", "ekg_findings-relativedate",<br/>"relativedate-external_body_part_or_region", "external_body_part_or_region-relativedate",<br/>"relativedate-fetus_newborn", "fetus_newborn-relativedate",<br/>"relativedate-hdl", "hdl-relativedate",<br/>"relativedate-heart_disease", "heart_disease-relativedate",<br/>"relativedate-height", "height-relativedate",<br/>"relativedate-hyperlipidemia", "hyperlipidemia-relativedate",<br/>"relativedate-hypertension", "hypertension-relativedate",<br/>"relativedate-imagingfindings", "imagingfindings-relativedate",<br/>"relativedate-imaging_technique", "imaging_technique-relativedate",<br/>"relativedate-injury_or_poisoning", "injury_or_poisoning-relativedate",<br/>"relativedate-internal_organ_or_component", "internal_organ_or_component-relativedate",<br/>"relativedate-kidney_disease", "kidney_disease-relativedate",<br/>"relativedate-ldl", "ldl-relativedate",<br/>"relativedate-modifier", "modifier-relativedate",<br/>"relativedate-o2_saturation", "o2_saturation-relativedate",<br/>"relativedate-obesity", "obesity-relativedate",<br/>"relativedate-oncological", "oncological-relativedate",<br/>"relativedate-overweight", "overweight-relativedate",<br/>"relativedate-oxygen_therapy", "oxygen_therapy-relativedate",<br/>"relativedate-pregnancy", "pregnancy-relativedate",<br/>"relativedate-procedure", "procedure-relativedate",<br/>"relativedate-psychological_condition", "psychological_condition-relativedate",<br/>"relativedate-pulse", "pulse-relativedate",<br/>"relativedate-respiration", "respiration-relativedate",<br/>"relativedate-smoking", "smoking-relativedate",<br/>"relativedate-substance", "substance-relativedate",<br/>"relativedate-substance_quantity", "substance_quantity-relativedate",<br/>"relativedate-symptom", "symptom-relativedate",<br/>"relativedate-temperature", "temperature-relativedate",<br/>"relativedate-test", "test-relativedate",<br/>"relativedate-test_result", "test_result-relativedate",<br/>"relativedate-total_cholesterol", "total_cholesterol-relativedate",<br/>"relativedate-treatment", "treatment-relativedate",<br/>"relativedate-triglycerides", "triglycerides-relativedate",<br/>"relativedate-vs_finding", "vs_finding-relativedate",<br/>"relativedate-vaccine", "vaccine-relativedate",<br/>"relativedate-vital_signs_header", "vital_signs_header-relativedate",<br/>"relativedate-weight", "weight-relativedate",<br/>"relativetime-admission_discharge", "admission_discharge-relativetime",<br/>"relativetime-alcohol", "alcohol-relativetime",<br/>"relativetime-allergen", "allergen-relativetime",<br/>"relativetime-bmi", "bmi-relativetime",<br/>"relativetime-birth_entity", "birth_entity-relativetime",<br/>"relativetime-blood_pressure", "blood_pressure-relativetime",<br/>"relativetime-cerebrovascular_disease", "cerebrovascular_disease-relativetime",<br/>"relativetime-clinical_dept", "clinical_dept-relativetime",<br/>"relativetime-communicable_disease", "communicable_disease-relativetime",<br/>"relativetime-death_entity", "death_entity-relativetime",<br/>"relativetime-diabetes", "diabetes-relativetime",<br/>"relativetime-diet", "diet-relativetime",<br/>"relativetime-disease_syndrome_disorder", "disease_syndrome_disorder-relativetime",<br/>"relativetime-drug_brandname", "drug_brandname-relativetime",<br/>"relativetime-drug_ingredient", "drug_ingredient-relativetime",<br/>"relativetime-ekg_findings", "ekg_findings-relativetime",<br/>"relativetime-external_body_part_or_region", "external_body_part_or_region-relativetime",<br/>"relativetime-fetus_newborn", "fetus_newborn-relativetime",<br/>"relativetime-hdl", "hdl-relativetime",<br/>"relativetime-heart_disease", "heart_disease-relativetime",<br/>"relativetime-height", "height-relativetime",<br/>"relativetime-hyperlipidemia", "hyperlipidemia-relativetime",<br/>"relativetime-hypertension", "hypertension-relativetime",<br/>"relativetime-imagingfindings", "imagingfindings-relativetime",<br/>"relativetime-imaging_technique", "imaging_technique-relativetime",<br/>"relativetime-injury_or_poisoning", "injury_or_poisoning-relativetime",<br/>"relativetime-internal_organ_or_component", "internal_organ_or_component-relativetime",<br/>"relativetime-kidney_disease", "kidney_disease-relativetime",<br/>"relativetime-ldl", "ldl-relativetime",<br/>"relativetime-modifier", "modifier-relativetime",<br/>"relativetime-o2_saturation", "o2_saturation-relativetime",<br/>"relativetime-obesity", "obesity-relativetime",<br/>"relativetime-oncological", "oncological-relativetime",<br/>"relativetime-overweight", "overweight-relativetime",<br/>"relativetime-oxygen_therapy", "oxygen_therapy-relativetime",<br/>"relativetime-pregnancy", "pregnancy-relativetime",<br/>"relativetime-procedure", "procedure-relativetime",<br/>"relativetime-psychological_condition", "psychological_condition-relativetime",<br/>"relativetime-pulse", "pulse-relativetime",<br/>"relativetime-respiration", "respiration-relativetime",<br/>"relativetime-smoking", "smoking-relativetime",<br/>"relativetime-substance", "substance-relativetime",<br/>"relativetime-substance_quantity", "substance_quantity-relativetime",<br/>"relativetime-symptom", "symptom-relativetime",<br/>"relativetime-temperature", "temperature-relativetime",<br/>"relativetime-test", "test-relativetime",<br/>"relativetime-test_result", "test_result-relativetime",<br/>"relativetime-total_cholesterol", "total_cholesterol-relativetime",<br/>"relativetime-treatment", "treatment-relativetime",<br/>"relativetime-triglycerides", "triglycerides-relativetime",<br/>"relativetime-vs_finding", "vs_finding-relativetime",<br/>"relativetime-vaccine", "vaccine-relativetime",<br/>"relativetime-vital_signs_header", "vital_signs_header-relativetime",<br/>"relativetime-weight", "weight-relativetime"] |